In [152]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf
import statsmodels.api as sm

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [153]:
df=pd.read_csv('final_estimation_file_apta_added_greg.csv',
                    encoding='utf-8')
df.drop(columns=['Unnamed: 0'],inplace=True)

In [154]:
tsd_data=pd.read_csv('tsd_pop_employment.csv',
                    encoding='utf-8')
tsd_data.drop(columns=['Unnamed: 0','CBSA Title'],inplace=True)
tsd_data.head()

,CBSA Code,STATE,Year,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10180,Texas,2002,57166.0,22735.0,25452.0,8979.0,0.0,0.0,0.0,48187.0,57166.0,0.0,0.0,9489.0,4531.0,3582.0,1376.0,0.0,0.0,0.0,8113.0,9489.0,0.0,0.0,58397.0,0.0,0.0,0.0,23086.0,25800.0,9511.0,48886.0,58397.0,0.0,0.0,1216.0,2355.0,0.0,0.0,0.0,954.0,1065.0,336.0,2019.0,2355.0,0.0,0.0
1,10180,Texas,2003,56917.0,21970.0,25046.0,9901.0,0.0,0.0,0.0,47016.0,56917.0,0.0,0.0,9484.0,4402.0,3471.0,1611.0,0.0,0.0,0.0,7873.0,9484.0,0.0,0.0,57221.0,0.0,0.0,0.0,22049.0,25074.0,10098.0,47123.0,57221.0,0.0,0.0,1216.0,2462.0,0.0,0.0,0.0,1032.0,1069.0,361.0,2101.0,2462.0,0.0,0.0
2,10180,Texas,2004,59025.0,22123.0,26375.0,10527.0,0.0,0.0,0.0,48498.0,59025.0,0.0,0.0,9599.0,4443.0,3594.0,1562.0,0.0,0.0,0.0,8037.0,9599.0,0.0,0.0,61419.0,0.0,0.0,0.0,23062.0,27199.0,11158.0,50261.0,61419.0,0.0,0.0,1216.0,2970.0,0.0,0.0,0.0,1231.0,1315.0,424.0,2546.0,2970.0,0.0,0.0
3,10180,Texas,2005,57879.0,21202.0,25353.0,11324.0,0.0,0.0,0.0,46555.0,57879.0,0.0,0.0,9645.0,4539.0,3500.0,1606.0,0.0,0.0,0.0,8039.0,9645.0,0.0,0.0,60001.0,0.0,0.0,0.0,21854.0,26081.0,12066.0,47935.0,60001.0,0.0,0.0,1216.0,2938.0,0.0,0.0,0.0,1134.0,1326.0,478.0,2460.0,2938.0,0.0,0.0
4,10180,Texas,2006,59588.0,21170.0,25759.0,12659.0,0.0,0.0,0.0,46929.0,59588.0,0.0,0.0,9819.0,4475.0,3516.0,1828.0,0.0,0.0,0.0,7991.0,9819.0,0.0,0.0,62311.0,0.0,0.0,0.0,22008.0,26639.0,13664.0,48647.0,62311.0,0.0,0.0,1216.0,2983.0,0.0,0.0,0.0,1122.0,1342.0,519.0,2464.0,2983.0,0.0,0.0


In [155]:
df=pd.merge(df,tsd_data,left_on=['CBSA','Year'],right_on=['CBSA Code','Year'],how='left')
df=df.drop_duplicates()
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,FARE_per_UPT_cleaned,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,PROBLEM_FLAG3,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,0.114148,2002,2018,0.114148,NaN,Bad fare,NaN,1.0,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,368095.58,5.96,94.04,1.36,NaN,119786.0,14.88,1.40,2014,http://www.crainscleveland.com/article/2014082...,0,0,4004892.28,35567.28438,36439.65938,37311.50938,60265.55938,77420.7875,1.9040,0,0,0,NaN,NaN,0.0,NaN,0,0,0,1,1,H,24,NaN,309488.2031,87.4,6.2,0.8,1.7,0.1,0.7,3.1,Goes to negative when extrapolating. Possible ...,0.159808,2417.239652,both,24,10420.0,Ohio,314749.0,96006.0,131687.0,87056.0,0.0,0.0,0.0,227693.0,314749.0,0.0,0.0,81463.0,23319.0,33326.0,24818.0,0.0,0.0,0.0,56645.0,81463.0,0.0,0.0,305401.0,0.0,0.0,0.0,92556.0,127165.0,85680.0,219721.0,305401.0,0.0,0.0,13619.2,54517.0,0.0,0.0,0.0,18228.0,25397.0,10892.0,43625.0,54517.0,0.0,0.0
1,10420,"Akron, OH Metro Area",Bus,2003,5395901.888,3976568.624,2.318587e+06,0.429694,2002,2018,0.429694,NaN,NaN,NaN,NaN,659397.0625,504631.8125,137582.3125,753.12500,25120.18750,25632.68750,25995.68750,10350.875,682850.56,8.81,8.66,82.79,24.33,63.0,12.68,26

In [5]:
list(tsd_data)

['CBSA Code',
 'STATE',
 'Year',
 'TOTAL_JOBS_WA_TRACTS',
 'JOBS_LESS_THAN_1250_WA_TRACTS',
 'JOBS_1250_3333_WA_TRACTS',
 'JOBS_MORE_THAN_3333_WA_TRACTS',
 'JOBS_WHITE_ALONE_WA_TRACTS',
 'JOBS_BLACK_ALONE_WA_TRACTS',
 'JOBS_HISPANIC_LATINO_WA_TRACTS',
 'JOBS_LESS_THAN_40K_WA_TRACTS',
 'JOBS_NON_WHITE_WA_TRACTS',
 'JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS',
 'JOBS_COLLEGE_OR_HIGHER_WA_TRACTS',
 'TOTAL_JOBS_WA_TSD_TRACTS',
 'JOBS_LESS_THAN_1250_WA_TSD_TRACTS',
 'JOBS_1250_3333_WA_TSD_TRACTS',
 'JOBS_MORE_THAN_3333_WA_TSD_TRACTS',
 'JOBS_WHITE_ALONE_WA_TSD_TRACTS',
 'JOBS_BLACK_ALONE_WA_TSD_TRACTS',
 'JOBS_HISPANIC_LATINO_WA_TSD_TRACTS',
 'JOBS_LESS_THAN_40K_WA_TSD_TRACTS',
 'JOBS_NON_WHITE_WA_TSD_TRACTS',
 'JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS',
 'JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS',
 'TOTAL_JOBS_RA_TRACTS',
 'TOTAL_JOBS_WHITE_ALONE_RA_TRACTS',
 'TOTAL_JOBS_BLACK_ALONE_RA_TRACTS',
 'TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS',
 'TOTAL_JOBS_UNDER_1250_RA_TRACTS',
 'TOTAL_JOBS_BETN_1250_3333_RA_TRACTS',

In [156]:
# WAC data derived columns. To account for people coming into the TSD from outlying areas.
df['TSD_WA_EMP_PCT_LEHD']=100*df['TOTAL_JOBS_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_PCT_CBSA']=100*df['TOTAL_JOBS_WA_TSD_TRACTS']/df['TOT_EMP_MSA']

df['JOBS_LESS_THAN_3333_WA_TRACTS']=df['JOBS_LESS_THAN_1250_WA_TRACTS']+df['JOBS_1250_3333_WA_TRACTS']
df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']=df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']+df['JOBS_1250_3333_WA_TSD_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['JOBS_LESS_THAN_1250_WA_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['JOBS_1250_3333_WA_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['JOBS_LESS_THAN_3333_WA_TRACTS']

df['TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']
df['TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['JOBS_1250_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']
df['TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD']=100*df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TSD_TRACTS']

#RAC data derived columns.

df['TSD_POP_PCT_LEHD']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS'] #RAC data
df['TSD_POP_PCT_CBSA']=100*df['TOTAL_JOBS_RA_TSD_TRACTS']/df['Tot_Pop']
df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_RA']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_1250_RA_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_RA']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_BETN_1250_3333_RA_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_RA']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_UNDER_3333_RA_TRACTS']

df['TSD_POP_WAGE_UNDER_1250_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_BETN_1250_3333_PCT_TSD']=100*df['TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_WAGE_UNDER_3333_PCT_TSD']=100*df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TSD_TRACTS']


In [159]:
# read updated TNC revenue data
# tnc_rev=pd.read_csv('TNC_REVENUE_3.csv')
# tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)
# tnc_rev=tnc_rev.drop_duplicates(['CBSA','MNAME','Year'])
# tnc_rev.head()

tnc_rev=pd.read_csv('TNC_CBSA_FINAL_2.csv')

tnc_rev.drop(columns=['MNAME','Tot_Pop'],inplace=True)
# tnc_rev=tnc_rev.drop_duplicates(['CBSA','MNAME','Year'])
tnc_rev.head()

,CBSA,Year,TNC_CLUSTER,TOT_POP_CLUSTER,TOTAL_YEARLY_POPULATION,POP_SHARE,TOTAL_TRIPS_PER_CAPITA,TNC_TRIPS_SCALED,TOTAL_TNC_TRIPS_PER_CLUSTER,TRIPS_SCALED,Scale_2,TNC_TRIPS,TNC_TRIPS_PER_CAPITA_CLUSTER
0,10420,2014,21,20063673.0,119360751,0.035997,4.034635,0.9718,80.9496,19.497887,0.514568,10.0,0.5
1,10740,2014,21,20063673.0,119360751,0.035997,4.034635,0.9718,80.9496,19.497887,0.514568,10.0,0.5
2,10900,2014,21,20063673.0,119360751,0.035997,4.034635,0.9718,80.9496,19.497887,0.514568,10.0,0.5
3,11460,2014,21,20063673.0,119360751,0.035997,4.034635,0.9718,80.9496,19.497887,0.514568,10.0,0.5
4,12420,2014,21,20063673.0,119360751,0.035997,4.034635,0.9718,80.9496,19.497887,0.514568,10.0,0.5


In [160]:
df=pd.merge(df,tnc_rev,on=['CBSA','Year'],how='left')
df['TNC_TRIPS_PER_CAPITA_CLUSTER']=df['TNC_TRIPS_PER_CAPITA_CLUSTER'].fillna(0)
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,FARE_per_UPT_cleaned,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,PROBLEM_FLAG3,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_WA_EMP_PCT_LEHD,TSD_WA_EMP_PCT_CBSA,JOBS_LESS_THAN_3333_WA_TRACTS,JOBS_LESS_THAN_3333_WA_TSD_TRACTS,TSD_WA_EMP_WAGE_UNDER_1250,TSD_WA_EMP_WAGE_BETN_1250_3333,TSD_WA_EMP_WAGE_UNDER_3333,TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD,TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,TNC_CLUSTER,TOT_POP_CLUSTER,TOTAL_YEARLY_POPULATION,POP_SHARE,TOTAL_TRIPS_PER_CAPITA,TNC_TRIPS_SCALED,TOTAL_TNC_TRIPS_PER_CLUSTER,TRIPS_SCALED,Scale_2,TNC_TRIPS,TNC_TRIPS_PER_CAPITA_CLUSTER
0,10420,"Akron, OH Metro Area",Bus,2002,5226385.660,4026372.388,5.965827e+05,0.114148,2002,2018,0.114148,NaN,Bad fare,NaN,1.0,646583.7344,496141.0469,137193.6719,-389.53125,25405.20313,26028.32813,26651.07813,7117.03125,682354.11,7.55,8.94,83.85,24.64,62.9,12.46,265452.20,22830.14,58254.52,117083.25,46891.53,20392.77,8.60,43046.82813,55300.5625,39.49,46.56,13.95,21948.42,346147.17,36809

In [161]:
df['TNC_TRIPS_PER_CAPITA_CLUSTER']=df['TNC_TRIPS_PER_CAPITA_CLUSTER'].astype(float)
df['TNC_TRIPS_PER_CAPITA_CLUSTER'].describe()

count    4718.000000
mean        0.841034
std         2.313638
min         0.000000
25%         0.000000
50%         0.000000
75%         0.700000
max        30.400000
Name: TNC_TRIPS_PER_CAPITA_CLUSTER, dtype: float64

In [162]:
df[df['CBSA']==14060]

,CBSA,MNAME,Mode,Year,UPT_ADJUSTED,VRM_ADJUSTED,FARE_ADJUSTED,FARE_per_UPT_cleaned,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,PROBLEM_FLAG3,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,AVG_FARE_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_WA_EMP_PCT_LEHD,TSD_WA_EMP_PCT_CBSA,JOBS_LESS_THAN_3333_WA_TRACTS,JOBS_LESS_THAN_3333_WA_TSD_TRACTS,TSD_WA_EMP_WAGE_UNDER_1250,TSD_WA_EMP_WAGE_BETN_1250_3333,TSD_WA_EMP_WAGE_UNDER_3333,TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD,TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,TNC_CLUSTER,TOT_POP_CLUSTER,TOTAL_YEARLY_POPULATION,POP_SHARE,TOTAL_TRIPS_PER_CAPITA,TNC_TRIPS_SCALED,TOTAL_TNC_TRIPS_PER_CLUSTER,TRIPS_SCALED,Scale_2,TNC_TRIPS,TNC_TRIPS_PER_CAPITA_CLUSTER
480,14060,"Bloomington-Normal, IL Metro Area",Bus,2006,1208833.0,875669.0,872238.0,0.721554,2006,2018,0.721554,1.0,Upt data discontinues,NaN,NaN,161202.00,118281.0,34191.00,1140.00,23640.00,22117.00,27353.00,52070,147507.00,12.70,7.00,80.3,22.5,67.6,9.9,61700.00,2937.00,20689.00,26726.0,8224.00,3124.0,4.76,51035.0,63714.0,34.90,46.10,19.00,2990.33,85829.67,88820.00,3.37,96.63,2.54,NaN,N

In [12]:
#df=pd.merge(df,tnc_rev,on=['CBSA','MNAME','Year'],how='left')
# df['TNC_REVENUE']=df['TNC_REVENUE'].fillna(0)
# df['TNC_TRIPS']=df['TNC_TRIPS'].fillna(0)
# df['TNC_BOOKINGS']=df['TNC_BOOKINGS'].fillna(0)
# df.head()

In [163]:
# read in and join the maintenance and restructure data
mr=pd.read_csv('maintenance_restructure.csv',
                    encoding='utf-8')
mr = mr[['CBSA', 'Mode', 'Year', 'MAINTENANCE', 'MAINTENANCE_NYC', 'MAINTENANCE_WMATA', 'RESTRUCTURE']]

df = df.merge(mr, on=['CBSA', 'Mode', 'Year'], how='left')

In [164]:
# read in mechanical and other failures

failures=pd.read_csv('maintenance_final.csv',encoding='utf-8')

failures.drop(columns=['Unnamed: 0'],inplace=True)

df=df.merge(failures, on=['CBSA','MNAME','Mode','Year'],how='left')

In [165]:
df=df.rename(columns={'UPT_ADJUSTED':'UPT_ADJ','VRM_ADJUSTED':'VRM_ADJ','FARE_ADJUSTED':'FARE_ADJ','AVG_FARE_2018':'FARE_per_UPT_2018',
                     'Mecha_Failures':'Mechanical_Failures'})

In [166]:
df['Mechanical_Failures']=df['Mechanical_Failures'].fillna(0)
df['Total_Failures']=df['Total_Failures'].fillna(0)
df['MDBF_Mechanical']=np.where(df['Mechanical_Failures']>0,df['VRM_ADJ']/df['Mechanical_Failures'],df['VRM_ADJ'])
df['MDBF_Total']=np.where(df['Total_Failures']>0,df['VRM_ADJ']/df['Total_Failures'],df['VRM_ADJ'])

In [167]:
# read in and join the bike share updates
bsu=pd.read_csv('bike_share_update.csv',
                    encoding='utf-8')
bsu = bsu[['CBSA', 'Mode', 'Year', 'PBS_Flag_Update']]

df = df.merge(bsu, on=['CBSA', 'Mode', 'Year'], how='left')

#df['PBS_Flag_Missing'] = df['PBS_Flag'].apply(lambda x : x.isnan())
df['PBS_Flag'] = np.where(df['PBS_Flag'].isnull(), df['PBS_Flag_Update'], df['PBS_Flag'])

In [168]:
# 4 apta clusers
df['CLUSTER_APTA4'] = np.floor(df['CLUSTER_APTA']/10)

In [169]:
# exclude if ridership is zero--missing data
df = df[df['UPT_ADJ']>0]

In [170]:
# fill in missing data as needed

# min fare is zero
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : max(x,0))

# max fare is $20
df['FARE_per_UPT_2018'] = df['FARE_per_UPT_2018'].apply(lambda x : min(x,20))

# zero vehicle HH can't go negative
df['HH_0Veh'] = df['HH_0Veh'].apply(lambda x : max(x,0))

In [171]:
df=df.drop_duplicates(['CBSA','Mode','Year'])

In [172]:
# exclude problematic data
old_len = len(df)
df = df[df['VRM_ADJ']>0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with missing VRM') 

# exclude if problem flag 3 is set
old_len = len(df)
df = df[df['PROBLEM_FLAG3']!=1]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with problem flag set to 1') 

# exclude rail in low op-ex group
old_len = len(df)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)
df['LOW_OPEX_RAIL'] = df['LOW_OPEX'] * df['RAIL_FLAG']
df = df[df['LOW_OPEX_RAIL']==0]
new_len = len(df)
print('Excluding ' + str(old_len-new_len) + ' records with that have rail in the low op-ex group') 



Excluding 1 records with missing VRM
Excluding 88 records with problem flag set to 1
Excluding 48 records with that have rail in the low op-ex group


In [173]:
# exclude discontinuous data
df = df.sort_values(by=['CBSA', 'Mode', 'Year'])

excluded_records = 100
while excluded_records > 0: 
    old_len = len(df)
    df['YearDiff'] = df['Year'].shift(-1) - df['Year']
    df = df[(df['YearDiff']==1) | (df['Year']==2018)]
    excluded_records = old_len - len(df)
    print('Excluding ' + str(excluded_records) + ' records with discontinuities in year.') 




Excluding 13 records with discontinuities in year.
Excluding 11 records with discontinuities in year.
Excluding 11 records with discontinuities in year.
Excluding 11 records with discontinuities in year.
Excluding 10 records with discontinuities in year.
Excluding 9 records with discontinuities in year.
Excluding 8 records with discontinuities in year.
Excluding 6 records with discontinuities in year.
Excluding 6 records with discontinuities in year.
Excluding 5 records with discontinuities in year.
Excluding 5 records with discontinuities in year.
Excluding 4 records with discontinuities in year.
Excluding 3 records with discontinuities in year.
Excluding 2 records with discontinuities in year.
Excluding 1 records with discontinuities in year.
Excluding 1 records with discontinuities in year.
Excluding 0 records with discontinuities in year.


In [174]:
df['BUS_FLAG'] = np.where(df['Mode']=='Bus', 1, 0)
df['RAIL_FLAG'] = np.where(df['Mode']=='Rail', 1, 0)

In [26]:
df.head()

,CBSA,MNAME,Mode,Year,UPT_ADJ,VRM_ADJ,FARE_ADJ,FARE_per_UPT_cleaned,min_year,max_year,FARE_per_UPT,PROBLEM_FLAG,NOTE,PROBLEM_FLAG2,PROBLEM_FLAG3,Tot_Pop_x,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Native_Outcountry_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,POP_CENSUSTRACT,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,Link_x,TNC_FLAG,YEARS_SINCE_TNC,FARE_ADJ_2018,TOTAL_MED_INC_INDIV_2018,NATIVE_INSTATE_MED_INC_2018,NATIVE_OUTSTATE_MED_INC_2018,HH_MED_INC_2018,HH_MEAN_INC_2018,GAS_PRICE_2018,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,Link,walkscore,transit,bike,CLUSTER_GT_NEW_11,CLUSTER_GT_ORIGINAL,CLUSTER_GT_8_GROUPS,CLUSTER_APTA,CLUSTER_NOTE,TOTAL_EMPLOYMENT,JTW_DA_PCT,JTW_CARPOOLED_PCT,JTW_TRANSIT_PCT,JTW_WALK_PCT,JTW_BICYCLE_PCT,JTW_OTHER_PCT,JTW_HOME_PCT,NOTE_JTW,FARE_per_UPT_2018,WEIGHTED_POP_DENSITY,_merge,CLUSTER_APTA_EXTENDED,CBSA Code,STATE,TOTAL_JOBS_WA_TRACTS,JOBS_LESS_THAN_1250_WA_TRACTS,JOBS_1250_3333_WA_TRACTS,JOBS_MORE_THAN_3333_WA_TRACTS,JOBS_WHITE_ALONE_WA_TRACTS,JOBS_BLACK_ALONE_WA_TRACTS,JOBS_HISPANIC_LATINO_WA_TRACTS,JOBS_LESS_THAN_40K_WA_TRACTS,JOBS_NON_WHITE_WA_TRACTS,JOBS_UPTO_HIGH_SCHOOL_WA_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TRACTS,TOTAL_JOBS_WA_TSD_TRACTS,JOBS_LESS_THAN_1250_WA_TSD_TRACTS,JOBS_1250_3333_WA_TSD_TRACTS,JOBS_MORE_THAN_3333_WA_TSD_TRACTS,JOBS_WHITE_ALONE_WA_TSD_TRACTS,JOBS_BLACK_ALONE_WA_TSD_TRACTS,JOBS_HISPANIC_LATINO_WA_TSD_TRACTS,JOBS_LESS_THAN_40K_WA_TSD_TRACTS,JOBS_NON_WHITE_WA_TSD_TRACTS,JOBS_UPTO_HIGH_SCHHOL_WA_TSD_TRACTS,JOBS_COLLEGE_OR_HIGHER_WA_TSD_TRACTS,TOTAL_JOBS_RA_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TRACTS,TOTAL_JOBS_OVER_3333_RA_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TRACTS,AREA_ACRE,TOTAL_JOBS_RA_TSD_TRACTS,TOTAL_JOBS_WHITE_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_BLACK_ALONE_RA_TSD_TRACTS,TOTAL_JOBS_HISPANIC_LATINO_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS,TOTAL_JOBS_BETN_1250_3333_RA_TSD_TRACTS,TOTAL_JOBS_OVER_3333_RA_TSD_TRACTS,TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS,TOTAL_JOBS_NON_WHITE_RA_TSD_TRACTS,TOTAL_JOBS_UPTO_HIGH_SCHHOL_RA_TSD_TRACTS,TOTAL_JOBS_COLLEGE_OR_HIGHER_RA_TSD_TRACTS,TSD_WA_EMP_PCT_LEHD,TSD_WA_EMP_PCT_CBSA,JOBS_LESS_THAN_3333_WA_TRACTS,JOBS_LESS_THAN_3333_WA_TSD_TRACTS,TSD_WA_EMP_WAGE_UNDER_1250,TSD_WA_EMP_WAGE_BETN_1250_3333,TSD_WA_EMP_WAGE_UNDER_3333,TSD_WA_EMP_WAGE_UNDER_1250_PCT_WA,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_3333_PCT_WA,TSD_WA_EMP_WAGE_UNDER_1250_PCT_TSD,TSD_WA_EMP_WAGE_BETN_1250_3333_PCT_TSD,TSD_WA_EMP_WAGE_UNDER_3333_PCT_TSD,TSD_POP_PCT_LEHD,TSD_POP_PCT_CBSA,TSD_POP_WAGE_UNDER_1250,TSD_POP_WAGE_BETN_1250_3333,TSD_POP_WAGE_UNDER_3333,TSD_POP_WAGE_UNDER_1250_PCT_RA,TSD_POP_WAGE_BETN_1250_3333_PCT_RA,TSD_POP_WAGE_UNDER_3333_PCT_RA,TSD_POP_WAGE_UNDER_1250_PCT_TSD,TSD_POP_WAGE_BETN_1250_3333_PCT_TSD,TSD_POP_WAGE_UNDER_3333_PCT_TSD,Tot_Pop_y,TNC_CLUSTER,TOT_POP_CLUSTER,TOTAL_YEARLY_POPULATION,POP_SHARE,TNC_TRIPS_PER_CAPITA,TNC_TRIPS_SCALED,TOTAL_TNC_TRIPS_PER_CLUSTER,TOT_TRIPS,TRIPS_PER_CAPITA,MAINTENANCE,MAINTENANCE_NYC,MAINTENANCE_WMATA,RESTRUCTURE,Mechanical_Failures,Other_Failures,Total_Failures,MDBF_Mechanical,MDBF_Total,PBS_Flag_Update,CLUSTER_APTA4,LOW_OPEX,RAIL_FLAG,LOW_OPEX_RAIL,YearDiff,BUS_FLAG
1,10420,"Akron, OH Metro Area",Bus,2003,5395901.888,3976568.624,2318586.755,0.429694,2002,2018,0.429694,NaN,NaN,NaN,NaN,659397.0625,504631.8125,137582.3125,753.125,25120.1875,25632.6875,25995.6875,10350.875,

In [175]:
# include population + employment
df['POP_EMP'] = df['Tot_Pop'] + df['TOT_EMP_MSA']
df['HH_EMP'] = df['Total_HH'] + df['TOT_EMP_MSA']


# percent of pop born outside USA
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [176]:
df.to_csv('merged_estfile.csv')

In [177]:
# time effects
df['YEARS_SINCE_2002']  = df['Year'] - 2002

df['YEARS_2002_2010']  = df['Year'].apply(lambda x : min(x-2002, 8))
df['YEARS_AFTER_2010'] = df['Year'].apply(lambda x : max(x-2010, 0))

df['YEARS_2002_2014']  = df['Year'].apply(lambda x : min(x-2002, 12))
df['YEARS_AFTER_2014'] = df['Year'].apply(lambda x : max(x-2014, 0))


df['YEARS_SINCE_2002_BUS']  = df['YEARS_SINCE_2002'] * df['BUS_FLAG']
df['YEARS_SINCE_2002_RAIL'] = df['YEARS_SINCE_2002'] * df['RAIL_FLAG']

df['YEARS_2002_2010_BUS']   = df['YEARS_2002_2010'] * df['BUS_FLAG']
df['YEARS_AFTER_2010_BUS']  = df['YEARS_AFTER_2010']* df['BUS_FLAG']
df['YEARS_2002_2010_RAIL']  = df['YEARS_2002_2010'] * df['RAIL_FLAG']
df['YEARS_AFTER_2010_RAIL'] = df['YEARS_AFTER_2010']* df['RAIL_FLAG']

df['YEARS_2002_2014_BUS']   = df['YEARS_2002_2014'] * df['BUS_FLAG']
df['YEARS_AFTER_2014_BUS']  = df['YEARS_AFTER_2014']* df['BUS_FLAG']
df['YEARS_2002_2014_RAIL']  = df['YEARS_2002_2014'] * df['RAIL_FLAG']
df['YEARS_AFTER_2014_RAIL'] = df['YEARS_AFTER_2014']* df['RAIL_FLAG']

In [178]:
# more time effects

df['YEARS_AFTER_2012'] = df['Year'].apply(lambda x : max(x-2012, 0))

df['TNC_YEARS_AFTER_2012'] = df['YEARS_AFTER_2012'] * df['TNC_FLAG']

df['TNC_YEARS_AFTER_2012_BUS'] = df['TNC_YEARS_AFTER_2012'] * df['BUS_FLAG']
df['TNC_YEARS_AFTER_2012_RAIL'] = df['TNC_YEARS_AFTER_2012'] * df['RAIL_FLAG']

In [179]:

df['YEARS_SINCE_TNC_2012'] = np.minimum(df['YEARS_SINCE_TNC'], df['YEARS_AFTER_2012'])

df['YEARS_SINCE_TNC_2012_BUS'] = df['YEARS_SINCE_TNC_2012'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_2012_RAIL'] = df['YEARS_SINCE_TNC_2012'] * df['RAIL_FLAG']

In [180]:
# test rail ramp up period
df['YEARS_SINCE_RAIL_START'] = (df['Year'] - df['min_year']) * df['RAIL_FLAG']
df['YEARS_SINCE_RAIL_START'] = np.where(df['min_year']==2002, 0, df['YEARS_SINCE_RAIL_START'])

df['YEARS_SINCE_RAIL_START_1'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 1))
df['YEARS_SINCE_RAIL_START_2'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_RAIL_START_3'] = df['YEARS_SINCE_RAIL_START'].apply(lambda x : min(x, 3))



In [181]:
# calculate bus vs rail stuff

# various bus/rail specifics

df['TNC_FLAG_BUS'] = df['TNC_FLAG'] * df['BUS_FLAG']
df['TNC_FLAG_RAIL'] = df['TNC_FLAG'] * df['RAIL_FLAG']

df['YEARS_SINCE_TNC_BUS'] = df['YEARS_SINCE_TNC'] * df['BUS_FLAG']
df['YEARS_SINCE_TNC_RAIL'] = df['YEARS_SINCE_TNC'] * df['RAIL_FLAG']

df['dockless_flag'] = df['dockCt'].apply(lambda x : min(x,1))
df['scooter_flag']  = df['scooterCt'].apply(lambda x : min(x,1))

df['BIKE_SHARE'] = df['PBS_Flag'] + df['dockless_flag']
df['BIKE_SHARE'] = df['BIKE_SHARE'].apply(lambda x : min(x,1))


df['PBS_Flag_BUS']   = df['PBS_Flag']   * df['BUS_FLAG']
df['dockCt_BUS']     = df['dockCt']     * df['BUS_FLAG']
df['docklessCt_BUS'] = df['docklessCt'] * df['BUS_FLAG']
df['scooterCt_BUS']  = df['scooterCt']  * df['BUS_FLAG']
df['dockless_flag_BUS'] = df['dockless_flag'] * df['BUS_FLAG']
df['scooter_flag_BUS']  = df['scooter_flag']  * df['BUS_FLAG']
df['BIKE_SHARE_BUS']  = df['BIKE_SHARE']  * df['BUS_FLAG']


df['PBS_Flag_RAIL']   = df['PBS_Flag']   * df['RAIL_FLAG']
df['dockCt_RAIL']     = df['dockCt']     * df['RAIL_FLAG']
df['docklessCt_RAIL'] = df['docklessCt'] * df['RAIL_FLAG']
df['scooterCt_RAIL']  = df['scooterCt']  * df['RAIL_FLAG']
df['dockless_flag_RAIL'] = df['dockless_flag'] * df['RAIL_FLAG']
df['scooter_flag_RAIL']  = df['scooter_flag']  * df['RAIL_FLAG']
df['BIKE_SHARE_RAIL']  = df['BIKE_SHARE']  * df['RAIL_FLAG']

In [182]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('#VALUE!', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('#VALUE!', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('#VALUE!', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('#VALUE!', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('#VALUE!', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('#VALUE!', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('#VALUE!', np.nan)


df['JTW_DA_PCT']        = df['JTW_DA_PCT'].replace('N', np.nan)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].replace('N', np.nan)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].replace('N', np.nan)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].replace('N', np.nan)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].replace('N', np.nan)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].replace('N', np.nan)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].replace('N', np.nan)



In [183]:
# convert JTW to numeric columns
df['JTW_DA_PCT']        = df['JTW_DA_PCT'].astype(float)
df['JTW_CARPOOLED_PCT'] = df['JTW_CARPOOLED_PCT'].astype(float)
df['JTW_TRANSIT_PCT']   = df['JTW_TRANSIT_PCT'].astype(float)
df['JTW_WALK_PCT']      = df['JTW_WALK_PCT'].astype(float)
df['JTW_BICYCLE_PCT']   = df['JTW_BICYCLE_PCT'].astype(float)
df['JTW_OTHER_PCT']     = df['JTW_OTHER_PCT'].astype(float)
df['JTW_HOME_PCT']      = df['JTW_HOME_PCT'].astype(float)

In [184]:
# immigrant population percentage
df['Tot_NonUSA_POP_pct'] = df['Tot_NonUSA_POP'] / df['Tot_Pop'] * 100

In [185]:
# bike share plus scooters for rail
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# different spec of bike and scooter
df['BIKE_SCOOTER_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] + df['scooter_flag_RAIL']

# merge bike and scooter for bus
df['BIKE_SCOOTER_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] + df['scooter_flag_BUS']

# merge bike and scooter for bus
df['BIKE_SCOOTER_BUS'] = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 

# scooter or bike
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_SHARE_BUS'] + df['scooter_flag_BUS'] 
df['BIKE_OR_SCOOTER_BUS']  = df['BIKE_OR_SCOOTER_BUS'].apply(lambda x : min(x, 1))

df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_SHARE_RAIL'] + df['scooter_flag_RAIL'] 
df['BIKE_OR_SCOOTER_RAIL'] = df['BIKE_OR_SCOOTER_RAIL'].apply(lambda x : min(x, 1))

# calculate average vehicles
df['AVG_VEHS'] = (df['HH_1Veh'] + 2*df['HH_2Veh'] + 3*df['HH_3Veh'] + 4*df['HH_4+Veh']) / (df['HH_0Veh'] + df['HH_1Veh'] + df['HH_2Veh'] + df['HH_3Veh'] + df['HH_4+Veh'])
df['AVG_VEHS_log'] = np.log(1+df['AVG_VEHS'])

In [186]:
# walk bike other commuting
df['JTW_WALK_BIKE_OTHER_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT'] + df['JTW_OTHER_PCT']

# walk bike commuting
df['JTW_WALK_BIKE_PCT'] = df['JTW_WALK_PCT'] + df['JTW_BICYCLE_PCT']

In [187]:
df['VRM_ADJ_BUS'] = df['VRM_ADJ'] * df['BUS_FLAG']
df['VRM_ADJ_RAIL'] = df['VRM_ADJ'] * df['RAIL_FLAG']


#maintenance effect on competing mode
df['MDBF_Mechanical_RAIL']  = df['MDBF_Mechanical'] * df['RAIL_FLAG']
df['MDBF_Mechanical_BUS'] = df['MDBF_Mechanical']  * df['BUS_FLAG']

df['MDBF_Total_RAIL']  = df['MDBF_Total'] * df['RAIL_FLAG']
df['MDBF_Total_BUS'] = df['MDBF_Total']  * df['BUS_FLAG']

In [188]:
# segment TNCs by cluster
df['HI_OPEX']  = np.where(df['CLUSTER_APTA4']==1, 1, 0)
df['MID_OPEX'] = np.where(df['CLUSTER_APTA4']==2, 1, 0)
df['LOW_OPEX'] = np.where(df['CLUSTER_APTA4']==3, 1, 0)
df['NEW_YORK'] = np.where(df['CLUSTER_APTA4']==10, 1, 0)

df['YEARS_SINCE_TNC_BUS_HI']  = df['YEARS_SINCE_TNC_BUS'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS_MID'] = df['YEARS_SINCE_TNC_BUS'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS_LOW'] = df['YEARS_SINCE_TNC_BUS'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_BUS_NY']  = df['YEARS_SINCE_TNC_BUS'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_RAIL_HI']  = df['YEARS_SINCE_TNC_RAIL'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL_MID'] = df['YEARS_SINCE_TNC_RAIL'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL_LOW'] = df['YEARS_SINCE_TNC_RAIL'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_RAIL_NY']  = df['YEARS_SINCE_TNC_RAIL'] * df['NEW_YORK']

df['YEARS_SINCE_TNC_BUS_HINY']  = df['YEARS_SINCE_TNC_BUS_HI'] + df['YEARS_SINCE_TNC_BUS_NY']
df['YEARS_SINCE_TNC_RAIL_HINY']  = df['YEARS_SINCE_TNC_RAIL_HI'] + df['YEARS_SINCE_TNC_RAIL_NY']

df['YEARS_SINCE_TNC_BUS_MIDLOW']  = df['YEARS_SINCE_TNC_BUS_MID'] + df['YEARS_SINCE_TNC_BUS_LOW']
df['YEARS_SINCE_TNC_RAIL_MIDLOW']  = df['YEARS_SINCE_TNC_RAIL_MID'] + df['YEARS_SINCE_TNC_RAIL_LOW']

In [189]:
# bikes and scooters by segment

df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']
df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']

df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']
df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']


df['BIKE_SHARE_HINY']  = df['BIKE_SHARE_HI'] + df['BIKE_SHARE_NY']
df['BIKE_SHARE_MIDLOW']  = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_HINY']  = df['scooter_flag_HI'] + df['scooter_flag_NY']
df['scooter_flag_MIDLOW']  = df['scooter_flag_MID'] + df['scooter_flag_LOW']

In [37]:
# tnc_rev=pd.read_csv('tnc_revenue.csv',encoding='utf-8')
# tnc_rev.head()

In [38]:
# tnc_rev.drop(columns=['Unnamed: 0'],inplace=True)

In [39]:
# df=pd.merge(df,tnc_rev,on=['CBSA',
#  'MNAME',
#  'Mode',
#  'Year',
#  'TNC_FLAG',
#  'YEARS_SINCE_TNC'],how='left')
# df.head()

In [40]:
# df['UBER_NET_REVENUE']=df['UBER_NET_REVENUE'].fillna(0)
# df['UBER_REV_SCALED']=df['UBER_REV_SCALED'].fillna(0)
# df.head(20)

In [41]:
# df['UBER_REV_SCALED_BUS'] = df['UBER_REV_SCALED'] * df['BUS_FLAG']
# df['UBER_REV_SCALED_RAIL'] = df['UBER_REV_SCALED'] * df['RAIL_FLAG']

# df['UBER_REV_SCALED_BUS_HI']  = df['UBER_REV_SCALED_BUS'] * df['HI_OPEX']
# df['UBER_REV_SCALED_BUS_MID'] = df['UBER_REV_SCALED_BUS'] * df['MID_OPEX']
# df['UBER_REV_SCALED_BUS_LOW'] = df['UBER_REV_SCALED_BUS'] * df['LOW_OPEX']
# df['UBER_REV_SCALED_BUS_NY']  = df['UBER_REV_SCALED_BUS'] * df['NEW_YORK']

# df['UBER_REV_SCALED_RAIL_HI']  = df['UBER_REV_SCALED_RAIL'] * df['HI_OPEX']
# df['UBER_REV_SCALED_RAIL_MID'] = df['UBER_REV_SCALED_RAIL'] * df['MID_OPEX']
# df['UBER_REV_SCALED_RAIL_LOW'] = df['UBER_REV_SCALED_RAIL'] * df['LOW_OPEX']
# df['UBER_REV_SCALED_RAIL_NY']  = df['UBER_REV_SCALED_RAIL'] * df['NEW_YORK']

# df['UBER_REV_SCALED_BUS_HINY']  = df['UBER_REV_SCALED_BUS_HI'] + df['UBER_REV_SCALED_BUS_NY']
# df['UBER_REV_SCALED_RAIL_HINY']  = df['UBER_REV_SCALED_RAIL_HI'] + df['UBER_REV_SCALED_RAIL_NY']

# df['UBER_REV_SCALED_BUS_MIDLOW']  = df['UBER_REV_SCALED_BUS_MID'] + df['UBER_REV_SCALED_BUS_LOW']
# df['UBER_REV_SCALED_RAIL_MIDLOW']  = df['UBER_REV_SCALED_RAIL_MID'] + df['UBER_REV_SCALED_RAIL_LOW']

In [42]:
# df['UBER_NET_REVENUE_BUS'] = df['UBER_NET_REVENUE'] * df['BUS_FLAG']
# df['UBER_NET_REVENUE_RAIL'] = df['UBER_NET_REVENUE'] * df['RAIL_FLAG']

# df['UBER_NET_REVENUE_BUS_HI']  = df['UBER_NET_REVENUE_BUS'] * df['HI_OPEX']
# df['UBER_NET_REVENUE_BUS_MID'] = df['UBER_NET_REVENUE_BUS'] * df['MID_OPEX']
# df['UBER_NET_REVENUE_BUS_LOW'] = df['UBER_NET_REVENUE_BUS'] * df['LOW_OPEX']
# df['UBER_NET_REVENUE_BUS_NY']  = df['UBER_NET_REVENUE_BUS'] * df['NEW_YORK']

# df['UBER_NET_REVENUE_RAIL_HI']  = df['UBER_NET_REVENUE_RAIL'] * df['HI_OPEX']
# df['UBER_NET_REVENUE_RAIL_MID'] = df['UBER_NET_REVENUE_RAIL'] * df['MID_OPEX']
# df['UBER_NET_REVENUE_RAIL_LOW'] = df['UBER_NET_REVENUE_RAIL'] * df['LOW_OPEX']
# df['UBER_NET_REVENUE_RAIL_NY']  = df['UBER_NET_REVENUE_RAIL'] * df['NEW_YORK']

# df['UBER_NET_REVENUE_BUS_HINY']  = df['UBER_NET_REVENUE_BUS_HI'] + df['UBER_NET_REVENUE_BUS_NY']
# df['UBER_NET_REVENUE_RAIL_HINY']  = df['UBER_NET_REVENUE_RAIL_HI'] + df['UBER_NET_REVENUE_RAIL_NY']

# df['UBER_NET_REVENUE_BUS_MIDLOW']  = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_LOW']
# df['UBER_NET_REVENUE_RAIL_MIDLOW']  = df['UBER_NET_REVENUE_RAIL_MID'] + df['UBER_NET_REVENUE_RAIL_LOW']

In [62]:
list(df)

['CBSA',
 'MNAME',
 'Mode',
 'Year',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'NOTE',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Native_Outcountry_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'Link_x',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018

In [190]:
#columns=['TNC_REVENUE','TNC_TRIPS','TNC_BOOKINGS']
columns=['TNC_TRIPS_PER_CAPITA_CLUSTER']
dummy=['HI_OPEX','MID_OPEX','LOW_OPEX','NEW_YORK']

for col in columns:
    df[col+'_BUS']=df[col]*df['BUS_FLAG']
    df[col+'_RAIL']=df[col]*df['RAIL_FLAG']
    
    for name in dummy:
        df[col+'_BUS_'+name]  = df[col]*df['BUS_FLAG'] * df[name]
        df[col+'_RAIL_'+name]  = df[col]*df['RAIL_FLAG'] * df[name]
#         df[col+'_BUS_MID'] = df['UBER_NET_REVENUE_BUS'] * df['MID_OPEX']
#         df[col+'_BUS_LOW'] = df['UBER_NET_REVENUE_BUS'] * df['LOW_OPEX']
#         df[col+'_BUS_NY']  = df['UBER_NET_REVENUE_BUS'] * df['NEW_YORK']

#         df['TNC_REVENUE_RAIL_HI']  = df['UBER_NET_REVENUE_RAIL'] * df['HI_OPEX']
#         df['TNC_REVENUE_RAIL_MID'] = df['UBER_NET_REVENUE_RAIL'] * df['MID_OPEX']
#         df['TNC_REVENUE_RAIL_LOW'] = df['UBER_NET_REVENUE_RAIL'] * df['LOW_OPEX']
#         df['TNC_REVENUE_RAIL_NY']  = df['UBER_NET_REVENUE_RAIL'] * df['NEW_YORK']

    df[col+'_BUS_HINY']  = df[col+'_BUS_HI_OPEX'] + df[col+'_BUS_NEW_YORK']
    df[col+'_BUS_MIDLOW']  = df[col+'_BUS_MID_OPEX'] + df[col+'_BUS_LOW_OPEX']
    df[col+'_BUS_HINY']  = df[col+'_BUS_HI_OPEX'] + df[col+'_BUS_NEW_YORK']
    
    df[col+'_RAIL_HINY']  = df[col+'_RAIL_HI_OPEX'] + df[col+'_RAIL_NEW_YORK']
    df[col+'_RAIL_MIDLOW']  = df[col+'_RAIL_MID_OPEX'] + df[col+'_RAIL_LOW_OPEX']
    df[col+'_RAIL_HINY']  = df[col+'_RAIL_HI_OPEX'] + df[col+'_RAIL_NEW_YORK']

# df['TNC_REVENUE_BUS_HINY']  = df['TNC_REVENUE_BUS_HI'] + df['TNC_REVENUE_BUS_NY']
# df['TNC_REVENUE_RAIL_HINY']  = df['TNC_REVENUE_RAIL_HI'] + df['UBER_NET_REVENUE_RAIL_NY']

# df['TNC_REVENUE_BUS_MIDLOW']  = df['UBER_NET_REVENUE_BUS_MID'] + df['UBER_NET_REVENUE_BUS_LOW']
# df['TNC_REVENUE_RAIL_MIDLOW']  = df['UBER_NET_REVENUE_RAIL_MID'] + df['UBER_NET_REVENUE_RAIL_LOW']

In [191]:
# set the indices
df['ID'] = df['MNAME'] + '-' + df['Mode']
df=df.set_index(['ID','Year'])

In [192]:
# keep only the numeric columns -- the estimation will give an error otherwise
df = df.select_dtypes(include=[np.number])

In [193]:
# create a log of all fields
for col in df.columns:
    df[col+'_log'] = np.log(df[col]+1)

In [194]:
# Test TNC ramp up

df['YEARS_SINCE_TNC_BUS_2'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_BUS_3'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_BUS_4'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_BUS_5'] = df['YEARS_SINCE_TNC_BUS'].apply(lambda x : min(x, 5))

df['YEARS_SINCE_TNC_RAIL_2'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 2))
df['YEARS_SINCE_TNC_RAIL_3'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 3))
df['YEARS_SINCE_TNC_RAIL_4'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 4))
df['YEARS_SINCE_TNC_RAIL_5'] = df['YEARS_SINCE_TNC_RAIL'].apply(lambda x : min(x, 5))

In [195]:
# count the first year of TNC presence
df['YEARS_SINCE_TNC_BUS2'] = df['YEARS_SINCE_TNC_BUS'] + df['TNC_FLAG_BUS']
df['YEARS_SINCE_TNC_RAIL2'] = df['YEARS_SINCE_TNC_RAIL'] + df['TNC_FLAG_RAIL']


In [196]:
## start in first year TNCs show up

df['YEARS_SINCE_TNC_BUS2_NY']  = df['YEARS_SINCE_TNC_BUS2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_BUS2_HI']  = df['YEARS_SINCE_TNC_BUS2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS2_MID'] = df['YEARS_SINCE_TNC_BUS2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS2_LOW'] = df['YEARS_SINCE_TNC_BUS2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_BUS2_HINY']   = df['YEARS_SINCE_TNC_BUS2_NY'] + df['YEARS_SINCE_TNC_BUS2_HI']
df['YEARS_SINCE_TNC_BUS2_MIDLOW'] = df['YEARS_SINCE_TNC_BUS2_MID'] + df['YEARS_SINCE_TNC_BUS2_LOW']

df['YEARS_SINCE_TNC_RAIL2_NY']  = df['YEARS_SINCE_TNC_RAIL2'] * df['NEW_YORK']
df['YEARS_SINCE_TNC_RAIL2_HI']  = df['YEARS_SINCE_TNC_RAIL2'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_RAIL2_MID'] = df['YEARS_SINCE_TNC_RAIL2'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_RAIL2_LOW'] = df['YEARS_SINCE_TNC_RAIL2'] * df['LOW_OPEX']

df['YEARS_SINCE_TNC_RAIL2_HINY']   = df['YEARS_SINCE_TNC_RAIL2_NY'] + df['YEARS_SINCE_TNC_RAIL2_HI']
df['YEARS_SINCE_TNC_RAIL2_MIDLOW'] = df['YEARS_SINCE_TNC_RAIL2_MID'] + df['YEARS_SINCE_TNC_RAIL2_LOW']

In [197]:
## BIKE SHARE SEGMENTATION

df['BIKE_SHARE_NY']  = df['BIKE_SHARE'] * df['NEW_YORK']
df['BIKE_SHARE_HI']  = df['BIKE_SHARE'] * df['HI_OPEX']
df['BIKE_SHARE_MID'] = df['BIKE_SHARE'] * df['MID_OPEX']
df['BIKE_SHARE_LOW'] = df['BIKE_SHARE'] * df['LOW_OPEX']

df['BIKE_SHARE_HINY']   = df['BIKE_SHARE_NY'] + df['BIKE_SHARE_HI']
df['BIKE_SHARE_MIDLOW'] = df['BIKE_SHARE_MID'] + df['BIKE_SHARE_LOW']

df['scooter_flag_NY']  = df['scooter_flag'] * df['NEW_YORK']
df['scooter_flag_HI']  = df['scooter_flag'] * df['HI_OPEX']
df['scooter_flag_MID'] = df['scooter_flag'] * df['MID_OPEX']
df['scooter_flag_LOW'] = df['scooter_flag'] * df['LOW_OPEX']

df['scooter_flag_HINY']   = df['scooter_flag_NY'] + df['scooter_flag_HI']
df['scooter_flag_MIDLOW'] = df['scooter_flag_MID'] + df['scooter_flag_LOW']



## Model Estimation

In [198]:
# base model 1 from 9-5
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + WEIGHTED_POP_DENSITY_log \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + EntityEffects \
                    + scooter_flag  \
                    ',data=df)
res2=mod.fit()
print(res2)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5624
Estimator:                   PanelOLS   R-squared (Between):              0.9380
No. Observations:                3967   R-squared (Within):               0.5624
Date:                Fri, May 08 2020   R-squared (Overall):              0.9381
Time:                        14:29:07   Log-likelihood                    2469.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      395.09
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.914   Distribution:                 F(12,3689)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             395.09
                            

### Transit Supportive Density

In [199]:
# switch to LEHD population percent

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5626
Estimator:                   PanelOLS   R-squared (Between):              0.9281
No. Observations:                3864   R-squared (Within):               0.5626
Date:                Fri, May 08 2020   R-squared (Overall):              0.9283
Time:                        14:29:22   Log-likelihood                    2417.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.26
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.26
                            

In [200]:
# try with LEHD wac data

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.9346
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.9349
Time:                        14:29:24   Log-likelihood                    2417.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.54
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.54
                            

In [201]:
# try with LEHD wac and rac data

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5636
Estimator:                   PanelOLS   R-squared (Between):              0.9338
No. Observations:                3864   R-squared (Within):               0.5636
Date:                Fri, May 08 2020   R-squared (Overall):              0.9340
Time:                        14:29:26   Log-likelihood                    2421.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      356.88
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(13,3593)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             356.88
                            

In [202]:
# add up RAC and WAC data. 
#RAC is the number of employed people living in a tract. the aggregation means the total number of people
#living in the TSD in an MSA
# WAC is the number of people working in a tract. the aggregation means that the total number of people employed
#in the TSD in an MSA
#Summing these two up, we get the total number of people living and working in the TSD of an MSA. 
#This value represents the intended transit customers in an MSA.

df['TSD_POP_EMP']=df['TOTAL_JOBS_WA_TSD_TRACTS']+df['TOTAL_JOBS_RA_TSD_TRACTS']
df['TSD_POP_EMP_TOTAL']=df['TOTAL_JOBS_WA_TRACTS']+df['TOTAL_JOBS_RA_TRACTS']
df['TSD_POP_EMP_PCT']=df['TSD_POP_EMP']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #reasonable coefficient on JTW_HOME_PCT
# use this one

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5625
Estimator:                   PanelOLS   R-squared (Between):              0.9319
No. Observations:                3864   R-squared (Within):               0.5625
Date:                Fri, May 08 2020   R-squared (Overall):              0.9322
Time:                        14:29:27   Log-likelihood                    2416.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.12
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.12
                            

In [203]:
#Testing the effect of the spread of low-wage population in the MSA

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

#Seems to be some kind of interaction between the two TSD variables.

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.9364
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.9367
Time:                        14:29:29   Log-likelihood                    2417.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      355.80
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(13,3593)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             355.80
                            

In [204]:
df['TSD_POP_EMP_WAGE_UNDER_1250']=df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']+df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']
df['TSD_POP_EMP_WAGE_UNDER_1250_PCT']=df['TSD_POP_EMP_WAGE_UNDER_1250']/df['TSD_POP_EMP_TOTAL']
df['TSD_POP_EMP_WAGE_UNDER_3333']=df['JOBS_LESS_THAN_3333_WA_TSD_TRACTS']+df['TOTAL_JOBS_UNDER_3333_RA_TSD_TRACTS']
df['TSD_POP_EMP_WAGE_UNDER_3333_PCT']=df['TSD_POP_EMP_WAGE_UNDER_3333']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.9389
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.9392
Time:                        14:29:31   Log-likelihood                    2418.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.57
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.57
                            

In [205]:
df['TSD_POP_EMP_WAGE_OVER_1250']=df['TSD_POP_EMP_TOTAL']-df['TSD_POP_EMP_WAGE_UNDER_1250']
df['TSD_POP_EMP_WAGE_OVER_1250_PCT']=df['TSD_POP_EMP_WAGE_OVER_1250']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + TSD_POP_EMP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.8678
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.8673
Time:                        14:29:33   Log-likelihood                    2418.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.57
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.017e+05
                            

In [206]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_3333_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5627
Estimator:                   PanelOLS   R-squared (Between):              0.9371
No. Observations:                3864   R-squared (Within):               0.5627
Date:                Fri, May 08 2020   R-squared (Overall):              0.9374
Time:                        14:29:36   Log-likelihood                    2417.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.40
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.40
                            

In [207]:
df['TSD_POP_EMP_WAGE_OVER_3333']=df['TSD_POP_EMP_TOTAL']-df['TSD_POP_EMP_WAGE_UNDER_3333']
df['TSD_POP_EMP_WAGE_OVER_3333_PCT']=df['TSD_POP_EMP_WAGE_OVER_3333']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_3333_PCT \
                    + TSD_POP_EMP_WAGE_OVER_3333_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5627
Estimator:                   PanelOLS   R-squared (Between):              0.8666
No. Observations:                3864   R-squared (Within):               0.5627
Date:                Fri, May 08 2020   R-squared (Overall):              0.8660
Time:                        14:29:38   Log-likelihood                    2417.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.40
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          7.606e+04
                            

In [208]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5626
Estimator:                   PanelOLS   R-squared (Between):              0.9353
No. Observations:                3864   R-squared (Within):               0.5626
Date:                Fri, May 08 2020   R-squared (Overall):              0.9355
Time:                        14:29:40   Log-likelihood                    2417.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.22
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.22
                            

In [209]:
#df['TSD_POP_WAGE_UNDER_1250']=100*df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']/df['TOTAL_JOBS_RA_TRACTS']
#df['TSD_WA_EMP_WAGE_UNDER_1250']=100*df['JOBS_LESS_THAN_1250_WA_TSD_TRACTS']/df['TOTAL_JOBS_WA_TRACTS']
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Looks the best one, but JTW goes the other way. Although JTW_HOME is still insignificant.

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.9365
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.9368
Time:                        14:29:43   Log-likelihood                    2417.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.53
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.53
                            

In [64]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [210]:
df['TSD_POP_WAGE_OVER_1250']=df['TOTAL_JOBS_RA_TRACTS']-df['TOTAL_JOBS_UNDER_1250_RA_TSD_TRACTS']
df['TSD_POP_WAGE_OVER_1250_PCT']=100*df['TSD_POP_WAGE_OVER_1250']/df['TOTAL_JOBS_RA_TRACTS']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.8660
No. Observations:                3864   R-squared (Within):               0.5628
Date:                Fri, May 08 2020   R-squared (Overall):              0.8655
Time:                        14:29:49   Log-likelihood                    2417.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.53
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             627.80
                            

In [211]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #We can use this one as the base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5621
Estimator:                   PanelOLS   R-squared (Between):              0.8610
No. Observations:                3872   R-squared (Within):               0.5621
Date:                Fri, May 08 2020   R-squared (Overall):              0.8607
Time:                        14:29:51   Log-likelihood                    2421.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      420.42
Entities:                         258   P-value                           0.0000
Avg Obs:                       15.008   Distribution:                 F(11,3603)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.374e+05
                            

## Years since TNC arrived

In [212]:
#base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5621
Estimator:                   PanelOLS   R-squared (Between):              0.8610
No. Observations:                3872   R-squared (Within):               0.5621
Date:                Fri, May 08 2020   R-squared (Overall):              0.8607
Time:                        14:29:55   Log-likelihood                    2421.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      420.42
Entities:                         258   P-value                           0.0000
Avg Obs:                       15.008   Distribution:                 F(11,3603)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.374e+05
                            

In [68]:
#TNC years after 2012 -- A little better

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #new base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5629
Estimator:                   PanelOLS   R-squared (Between):              0.8675
No. Observations:                3861   R-squared (Within):               0.5629
Date:                Wed, May 06 2020   R-squared (Overall):              0.8676
Time:                        11:43:17   Log-likelihood                    2417.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.51
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             628.12
                            

In [69]:
# exponentiated years since TNC

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + np.exp(YEARS_SINCE_TNC_BUS)\
                    + np.exp(YEARS_SINCE_TNC_RAIL) \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5570
Estimator:                   PanelOLS   R-squared (Between):              0.9172
No. Observations:                3861   R-squared (Within):               0.5570
Date:                Wed, May 06 2020   R-squared (Overall):              0.9176
Time:                        11:43:18   Log-likelihood                    2391.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      376.35
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             376.35
                            

# New TNC Trips

In [148]:
#base model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5625
Estimator:                   PanelOLS   R-squared (Between):              0.9319
No. Observations:                3864   R-squared (Within):               0.5625
Date:                Fri, May 08 2020   R-squared (Overall):              0.9322
Time:                        12:29:17   Log-likelihood                    2416.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.12
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             385.12
                            

In [215]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [213]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5592
Estimator:                   PanelOLS   R-squared (Between):              0.9218
No. Observations:                3864   R-squared (Within):               0.5592
Date:                Fri, May 08 2020   R-squared (Overall):              0.9222
Time:                        14:30:15   Log-likelihood                    2401.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      379.89
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(12,3594)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             379.89
                            

In [216]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HINY \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MIDLOW \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HINY \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5638
Estimator:                   PanelOLS   R-squared (Between):              0.9473
No. Observations:                3864   R-squared (Within):               0.5638
Date:                Fri, May 08 2020   R-squared (Overall):              0.9475
Time:                        14:33:18   Log-likelihood                    2422.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      331.64
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(14,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             331.64
                            

# Uber Bookings

In [70]:
# base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5629
Estimator:                   PanelOLS   R-squared (Between):              0.8675
No. Observations:                3861   R-squared (Within):               0.5629
Date:                Wed, May 06 2020   R-squared (Overall):              0.8676
Time:                        11:43:19   Log-likelihood                    2417.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.51
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             628.12
                            

In [71]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_log\
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5562
Estimator:                   PanelOLS   R-squared (Between):              0.8611
No. Observations:                3861   R-squared (Within):               0.5562
Date:                Wed, May 06 2020   R-squared (Overall):              0.8608
Time:                        11:43:20   Log-likelihood                    2387.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      409.41
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(11,3593)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             670.15
                            

In [72]:
#TNC bookings separated by bus and rail mode
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_BUS_log\
                    + TNC_BOOKINGS_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5570
Estimator:                   PanelOLS   R-squared (Between):              0.8610
No. Observations:                3861   R-squared (Within):               0.5570
Date:                Wed, May 06 2020   R-squared (Overall):              0.8608
Time:                        11:43:20   Log-likelihood                    2391.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      376.36
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             615.73
                            

In [73]:
#separate Bookings by different clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_BOOKINGS_BUS_HINY_log \
                    + TNC_BOOKINGS_BUS_MIDLOW_log \
                    + TNC_BOOKINGS_RAIL_HINY_log \
                    + TNC_BOOKINGS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #worse than Years Since TNC after 2012 variable

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5585
Estimator:                   PanelOLS   R-squared (Between):              0.8613
No. Observations:                3861   R-squared (Within):               0.5585
Date:                Wed, May 06 2020   R-squared (Overall):              0.8611
Time:                        11:43:22   Log-likelihood                    2397.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.42
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.066e+05
                            

# Uber trips

In [74]:
# new base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5629
Estimator:                   PanelOLS   R-squared (Between):              0.8675
No. Observations:                3861   R-squared (Within):               0.5629
Date:                Wed, May 06 2020   R-squared (Overall):              0.8676
Time:                        11:43:23   Log-likelihood                    2417.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.51
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             628.12
                            

In [75]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_log \
                    + TNC_TRIPS_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5571
Estimator:                   PanelOLS   R-squared (Between):              0.8613
No. Observations:                3861   R-squared (Within):               0.5571
Date:                Wed, May 06 2020   R-squared (Overall):              0.8611
Time:                        11:43:24   Log-likelihood                    2391.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      376.55
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             615.98
                            

In [76]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_HINY_log \
                    + TNC_TRIPS_RAIL_HINY_log \
                    + TNC_TRIPS_BUS_MIDLOW_log \
                    + TNC_TRIPS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5586
Estimator:                   PanelOLS   R-squared (Between):              0.8618
No. Observations:                3861   R-squared (Within):               0.5586
Date:                Wed, May 06 2020   R-squared (Overall):              0.8615
Time:                        11:43:25   Log-likelihood                    2398.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.53
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.066e+05
                            

In [217]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HINY \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MIDLOW \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HINY \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5638
Estimator:                   PanelOLS   R-squared (Between):              0.9473
No. Observations:                3864   R-squared (Within):               0.5638
Date:                Fri, May 08 2020   R-squared (Overall):              0.9475
Time:                        14:36:03   Log-likelihood                    2422.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      331.64
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(14,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             331.64
                            

In [218]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5650
Estimator:                   PanelOLS   R-squared (Between):              0.9497
No. Observations:                3864   R-squared (Within):               0.5650
Date:                Fri, May 08 2020   R-squared (Overall):              0.9499
Time:                        14:38:10   Log-likelihood                    2427.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      274.23
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(17,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             274.23
                            

In [219]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [220]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE_BUS \
                    + BIKE_SHARE_RAIL \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5654
Estimator:                   PanelOLS   R-squared (Between):              0.9485
No. Observations:                3864   R-squared (Within):               0.5654
Date:                Fri, May 08 2020   R-squared (Overall):              0.9487
Time:                        14:41:51   Log-likelihood                    2429.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      245.57
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(19,3587)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             245.57
                            

In [221]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE_BUS \
                    + BIKE_SHARE_RAIL \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5631
Estimator:                   PanelOLS   R-squared (Between):              0.9505
No. Observations:                3977   R-squared (Within):               0.5631
Date:                Fri, May 08 2020   R-squared (Overall):              0.9507
Time:                        14:43:00   Log-likelihood                    2440.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      264.38
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(18,3692)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             264.38
                            

In [222]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE_BUS \
                    + BIKE_SHARE_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5647
Estimator:                   PanelOLS   R-squared (Between):              0.9456
No. Observations:                3864   R-squared (Within):               0.5647
Date:                Fri, May 08 2020   R-squared (Overall):              0.9459
Time:                        14:43:56   Log-likelihood                    2426.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      273.84
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(17,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             273.84
                            

In [223]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)
#MAY 8

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5653
Estimator:                   PanelOLS   R-squared (Between):              0.9493
No. Observations:                3864   R-squared (Within):               0.5653
Date:                Fri, May 08 2020   R-squared (Overall):              0.9495
Time:                        14:45:00   Log-likelihood                    2428.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      259.20
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(18,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             259.20
                            

In [77]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_HINY_log \
                    + TNC_TRIPS_BUS_MID_OPEX_log \
                    + TNC_TRIPS_BUS_LOW_OPEX_log \
                    + TNC_TRIPS_RAIL_HINY_log \
                    + TNC_TRIPS_RAIL_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ', data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5598
Estimator:                   PanelOLS   R-squared (Between):              0.8597
No. Observations:                3861   R-squared (Within):               0.5598
Date:                Wed, May 06 2020   R-squared (Overall):              0.8595
Time:                        11:43:26   Log-likelihood                    2403.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      304.30
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          9.973e+04
                            

In [78]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

### Uber Scaled Revenue

In [79]:
# switch to Uber scaled revenue
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + TSD_POP_PCT_LEHD \
#                     + JTW_HOME_PCT \
#                     + UBER_REV_SCALED_BUS \
#                     + UBER_REV_SCALED_RAIL \
#                     + BIKE_SHARE \
#                     + scooter_flag  \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

In [80]:
# switch to Uber scaled revenue
# mod=PanelOLS.from_formula('UPT_ADJ_log \
#                     ~ VRM_ADJ_log \
#                     + FARE_per_UPT_2018_log \
#                     + POP_EMP_log \
#                     + GAS_PRICE_2018_log \
#                     + TOTAL_MED_INC_INDIV_2018_log \
#                     + PCT_HH_NO_VEH \
#                     + TSD_POP_PCT_LEHD \
#                     + JTW_HOME_PCT \
#                     + UBER_REV_SCALED_BUS_HINY \
#                     + UBER_REV_SCALED_BUS_MIDLOW \
#                     + UBER_REV_SCALED_RAIL_HINY \
#                     + UBER_REV_SCALED_RAIL_MIDLOW \
#                     + BIKE_SHARE \
#                     + scooter_flag  \
#                     + EntityEffects \
#                     ',data=df)
# res=mod.fit()
# print(res)

### Uber un-scaled revenue

In [81]:
# new base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_2012_BUS \
                    + YEARS_SINCE_TNC_2012_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5629
Estimator:                   PanelOLS   R-squared (Between):              0.8675
No. Observations:                3861   R-squared (Within):               0.5629
Date:                Wed, May 06 2020   R-squared (Overall):              0.8676
Time:                        11:43:31   Log-likelihood                    2417.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      385.51
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             628.12
                            

In [82]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS \
                    + TNC_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5576
Estimator:                   PanelOLS   R-squared (Between):              0.8564
No. Observations:                3861   R-squared (Within):               0.5576
Date:                Wed, May 06 2020   R-squared (Overall):              0.8561
Time:                        11:43:31   Log-likelihood                    2393.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      377.31
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             617.01
                            

In [83]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [84]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MIDLOW \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5657
Estimator:                   PanelOLS   R-squared (Between):              0.8634
No. Observations:                3861   R-squared (Within):               0.5657
Date:                Wed, May 06 2020   R-squared (Overall):              0.8634
Time:                        11:43:32   Log-likelihood                    2429.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      334.04
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             543.21
                            

In [85]:
# switch to Uber un-scaled revenue
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MID_OPEX \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Revenue by clusters performs better

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5664
Estimator:                   PanelOLS   R-squared (Between):              0.8675
No. Observations:                3861   R-squared (Within):               0.5664
Date:                Wed, May 06 2020   R-squared (Overall):              0.8677
Time:                        11:43:33   Log-likelihood                    2432.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      312.56
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             508.04
                            

In [86]:
# Use the logged transformed revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5601
Estimator:                   PanelOLS   R-squared (Between):              0.8606
No. Observations:                3861   R-squared (Within):               0.5601
Date:                Wed, May 06 2020   R-squared (Overall):              0.8604
Time:                        11:43:33   Log-likelihood                    2404.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      304.61
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          9.978e+04
                            

In [87]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_log \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5573
Estimator:                   PanelOLS   R-squared (Between):              0.8621
No. Observations:                3861   R-squared (Within):               0.5573
Date:                Wed, May 06 2020   R-squared (Overall):              0.8620
Time:                        11:43:34   Log-likelihood                    2392.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      376.75
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):           1.24e+05
                            

In [88]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HI_OPEX_log \
                    + TNC_REVENUE_RAIL_HI_OPEX_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + TNC_REVENUE_BUS_NEW_YORK_log \
                    + TNC_REVENUE_RAIL_NEW_YORK_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5606
Estimator:                   PanelOLS   R-squared (Between):              0.8621
No. Observations:                3861   R-squared (Within):               0.5606
Date:                Wed, May 06 2020   R-squared (Overall):              0.8620
Time:                        11:43:34   Log-likelihood                    2407.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      269.20
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(17,3587)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):           8.81e+04
                            

In [89]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5601
Estimator:                   PanelOLS   R-squared (Between):              0.8606
No. Observations:                3861   R-squared (Within):               0.5601
Date:                Wed, May 06 2020   R-squared (Overall):              0.8604
Time:                        11:43:35   Log-likelihood                    2404.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      304.61
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          9.978e+04
                            

In [90]:
df['TNC_REVENUE_HINY']=df['TNC_REVENUE']*df['HI_OPEX']+df['TNC_REVENUE']*df['NEW_YORK']
df['TNC_REVENUE_HINY_log']=np.log(df['TNC_REVENUE_HINY']+1)

In [91]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #Performs about the same as TNC Trips
#new base model

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5586
Estimator:                   PanelOLS   R-squared (Between):              0.8641
No. Observations:                3861   R-squared (Within):               0.5586
Date:                Wed, May 06 2020   R-squared (Overall):              0.8639
Time:                        11:43:36   Log-likelihood                    2398.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.46
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.066e+05
                            

In [92]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

# Per capita TNC trips

In [93]:
df['PER_CAPITA_TNC_TRIPS']=df['TNC_TRIPS']/df['Tot_Pop']

df['PER_CAPITA_TNC_TRIPS_HI_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['HI_OPEX']
df['PER_CAPITA_TNC_TRIPS_MID_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['MID_OPEX']
df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']=df['PER_CAPITA_TNC_TRIPS']*df['LOW_OPEX']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK']=df['PER_CAPITA_TNC_TRIPS']*df['NEW_YORK']
df['PER_CAPITA_TNC_TRIPS_HINY']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']+df['PER_CAPITA_TNC_TRIPS_NEW_YORK']
df['PER_CAPITA_TNC_TRIPS_MIDLOW']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']+df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']

df['PER_CAPITA_TNC_TRIPS_HINY_BUS']=df['PER_CAPITA_TNC_TRIPS_HINY']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_HINY_RAIL']=df['PER_CAPITA_TNC_TRIPS_HINY']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_MIDLOW_BUS']=df['PER_CAPITA_TNC_TRIPS_MIDLOW']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL']=df['PER_CAPITA_TNC_TRIPS_MIDLOW']*df['RAIL_FLAG']

df['PER_CAPITA_TNC_BOOKINGS']=df['TNC_BOOKINGS']/df['Tot_Pop']
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['HI_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['MID_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']=df['PER_CAPITA_TNC_BOOKINGS']*df['LOW_OPEX']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']=df['PER_CAPITA_TNC_BOOKINGS']*df['NEW_YORK']
df['PER_CAPITA_TNC_BOOKINGS_HINY']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']+df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']+df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']

df['PER_CAPITA_TNC_BOOKINGS_HINY_BUS']=df['PER_CAPITA_TNC_BOOKINGS_HINY']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_HINY_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_HINY']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW_BUS']=df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MIDLOW_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_MIDLOW']*df['RAIL_FLAG']


In [94]:
df['PER_CAPITA_TNC_TRIPS_BUS']=df['PER_CAPITA_TNC_TRIPS']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_RAIL']=df['PER_CAPITA_TNC_TRIPS']*df['RAIL_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5597
Estimator:                   PanelOLS   R-squared (Between):              0.8580
No. Observations:                3861   R-squared (Within):               0.5597
Date:                Wed, May 06 2020   R-squared (Overall):              0.8578
Time:                        11:43:38   Log-likelihood                    2403.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      380.52
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):          1.247e+05
                            

In [95]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5633
Estimator:                   PanelOLS   R-squared (Between):              0.8718
No. Observations:                3861   R-squared (Within):               0.5633
Date:                Wed, May 06 2020   R-squared (Overall):              0.8719
Time:                        11:43:39   Log-likelihood                    2418.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      330.75
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             538.76
                            

In [96]:
df['PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS']=df['PER_CAPITA_TNC_TRIPS_NEW_YORK']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL']=df['PER_CAPITA_TNC_TRIPS_NEW_YORK']*df['RAIL_FLAG']
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + TSD_POP_WAGE_UNDER_1250 + TSD_POP_WAGE_OVER_1250_PCT + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                                                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [97]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [98]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + TSD_POP_PCT_LEHD + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [99]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + TSD_POP_EMP_PCT + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [100]:
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']=df['TNC_TRIPS']/df['UPT_ADJ']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5574
Estimator:                   PanelOLS   R-squared (Between):              0.9177
No. Observations:                3861   R-squared (Within):               0.5574
Date:                Wed, May 06 2020   R-squared (Overall):              0.9180
Time:                        11:43:42   Log-likelihood                    2393.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      376.97
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(12,3592)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             376.97
                            

In [101]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + TSD_POP_PCT_LEHD + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [102]:
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_HI_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK_BUS']=df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_NEW_YORK']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX_RAIL']=df['PER_CAPITA_TNC_BOOKINGS_MID_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX_BUS']=df['PER_CAPITA_TNC_BOOKINGS_LOW_OPEX']*df['BUS_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5655
Estimator:                   PanelOLS   R-squared (Between):              0.9354
No. Observations:                3861   R-squared (Within):               0.5655
Date:                Wed, May 06 2020   R-squared (Overall):              0.9356
Time:                        11:43:44   Log-likelihood                    2428.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      274.61
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(17,3587)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             274.61
                            

In [103]:
df['PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']*df['BUS_FLAG']
df['PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL']=df['PER_CAPITA_TNC_TRIPS_MID_OPEX']*df['RAIL_FLAG']
df['PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS']=df['PER_CAPITA_TNC_TRIPS_LOW_OPEX']*df['BUS_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TSD_POP_WAGE_OVER_1250_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5651
Estimator:                   PanelOLS   R-squared (Between):              0.8703
No. Observations:                3861   R-squared (Within):               0.5651
Date:                Wed, May 06 2020   R-squared (Overall):              0.8704
Time:                        11:43:45   Log-likelihood                    2426.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      310.84
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             505.72
                            

In [104]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [105]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_BUS_HINY \
                    + TNC_REVENUE_BUS_MID_OPEX \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5652
Estimator:                   PanelOLS   R-squared (Between):              0.9127
No. Observations:                3869   R-squared (Within):               0.5652
Date:                Wed, May 06 2020   R-squared (Overall):              0.9130
Time:                        11:43:46   Log-likelihood                    2434.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      311.71
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(15,3597)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             311.71
                            

In [286]:
# New base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

#separate bus and rail

PatsyError: Error evaluating factor: NameError: name 'PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL' is not defined
    0 + VRM_ADJ_log + FARE_per_UPT_2018_log + POP_EMP_log + GAS_PRICE_2018_log + TOTAL_MED_INC_INDIV_2018_log + PCT_HH_NO_VEH + TSD_POP_PCT_LEHD + JTW_HOME_PCT + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL + BIKE_SHARE + scooter_flag
                                                                                                                                                                                                                                                                                                                                                                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# TNC Cost vs Transit Cost

In [107]:
# Test transit fare revenue against TNC fare revenue

df['TRANSIT_FARE_VS_TNC_FARE']=df['TNC_REVENUE']/(df['FARE_ADJ_2018'])
df['TNC_COST_PER_TRIP']=df['TNC_REVENUE']/df['TNC_TRIPS']
df['TNC_COST_VS_TRANSIT_FARE']=df['TNC_COST_PER_TRIP']/(df['FARE_per_UPT_2018'])

In [108]:
#base model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5586
Estimator:                   PanelOLS   R-squared (Between):              0.9254
No. Observations:                3861   R-squared (Within):               0.5586
Date:                Wed, May 06 2020   R-squared (Overall):              0.9258
Time:                        11:43:50   Log-likelihood                    2398.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.46
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             324.46
                            

In [109]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:1872: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


ValueError: exog does not have full column rank.

### Separate Rail vs bus models

In [110]:
#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) #USE THIS ONE

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5656
Estimator:                   PanelOLS   R-squared (Between):              0.9357
No. Observations:                3861   R-squared (Within):               0.5656
Date:                Wed, May 06 2020   R-squared (Overall):              0.9359
Time:                        11:43:58   Log-likelihood                    2428.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      274.68
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(17,3587)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             274.68
                            

In [111]:
df['PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS']=df['PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS']+df['PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS']+df['PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5595
Estimator:                   PanelOLS   R-squared (Between):              0.9124
No. Observations:                3861   R-squared (Within):               0.5595
Date:                Wed, May 06 2020   R-squared (Overall):              0.9128
Time:                        11:43:59   Log-likelihood                    2402.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      350.81
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(13,3591)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             350.81
                            

In [112]:
#bus model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5491
Estimator:                   PanelOLS   R-squared (Between):              0.9617
No. Observations:                3308   R-squared (Within):               0.5491
Date:                Wed, May 06 2020   R-squared (Overall):              0.9618
Time:                        11:44:03   Log-likelihood                    2426.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      312.41
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(12,3079)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             312.41
                            

In [113]:
#rail mode

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6381
Estimator:                   PanelOLS   R-squared (Between):              0.6274
No. Observations:                 553   R-squared (Within):               0.6381
Date:                Wed, May 06 2020   R-squared (Overall):              0.6347
Time:                        11:44:03   Log-likelihood                    167.28
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      80.449
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(11,502)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             80.449
                            

In [114]:
# separate bus model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5587
Estimator:                   PanelOLS   R-squared (Between):              0.9793
No. Observations:                3308   R-squared (Within):               0.5587
Date:                Wed, May 06 2020   R-squared (Overall):              0.9793
Time:                        11:44:04   Log-likelihood                    2462.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      278.28
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(14,3077)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             278.28
                            

In [115]:
#rail model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6386
Estimator:                   PanelOLS   R-squared (Between):              0.6463
No. Observations:                 553   R-squared (Within):               0.6386
Date:                Wed, May 06 2020   R-squared (Overall):              0.6531
Time:                        11:44:04   Log-likelihood                    167.71
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      67.968
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(13,500)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             67.968
                            

In [116]:
# separate bus model with logged transformed TNC revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5498
Estimator:                   PanelOLS   R-squared (Between):              0.9623
No. Observations:                3308   R-squared (Within):               0.5498
Date:                Wed, May 06 2020   R-squared (Overall):              0.9624
Time:                        11:44:05   Log-likelihood                    2429.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      289.17
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(13,3078)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             289.17
                            

In [117]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MIDLOW_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res) #separate coeff for mid and low clusters seems to be better

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5484
Estimator:                   PanelOLS   R-squared (Between):              0.9648
No. Observations:                3308   R-squared (Within):               0.5484
Date:                Wed, May 06 2020   R-squared (Overall):              0.9650
Time:                        11:44:05   Log-likelihood                    2424.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      311.55
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(12,3079)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             311.55
                            

In [118]:
#base model for bus
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects ', 
                    data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5498
Estimator:                   PanelOLS   R-squared (Between):              0.9623
No. Observations:                3308   R-squared (Within):               0.5498
Date:                Wed, May 06 2020   R-squared (Overall):              0.9624
Time:                        11:44:06   Log-likelihood                    2429.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      289.17
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(13,3078)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             289.17
                            

## Rail Model

In [119]:
# separate rail model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY \
                    + TNC_REVENUE_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) #TSD low wages concentration coefficient doesn't work

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6368
Estimator:                   PanelOLS   R-squared (Between):              0.6744
No. Observations:                 553   R-squared (Within):               0.6368
Date:                Wed, May 06 2020   R-squared (Overall):              0.6806
Time:                        11:44:07   Log-likelihood                    166.29
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.185
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.185
                            

In [120]:
# separate rail model with log transformed TNC revenue

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6378
Estimator:                   PanelOLS   R-squared (Between):              0.5632
No. Observations:                 553   R-squared (Within):               0.6378
Date:                Wed, May 06 2020   R-squared (Overall):              0.5730
Time:                        11:44:07   Log-likelihood                    167.08
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.514
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.514
                            

In [121]:
#TNC revenue together
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6331
Estimator:                   PanelOLS   R-squared (Between):              0.6096
No. Observations:                 553   R-squared (Within):               0.6331
Date:                Wed, May 06 2020   R-squared (Overall):              0.6175
Time:                        11:44:08   Log-likelihood                    163.51
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      78.742
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(11,502)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             78.742
                            

In [122]:
# For Rail centric MSAs, the concentration of low-wage population may not affect that much. 

# df['TSD_POP_EMP']=df['TOTAL_JOBS_WA_TSD_TRACTS']+df['TOTAL_JOBS_RA_TSD_TRACTS']
# df['TSD_POP_EMP_TOTAL']=df['TOTAL_JOBS_WA_TRACTS']+df['TOTAL_JOBS_RA_TRACTS']
# df['TSD_POP_EMP_PCT']=df['TSD_POP_EMP']/df['TSD_POP_EMP_TOTAL']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) #greater r-squared, but TSD variable still goes the other way

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6403
Estimator:                   PanelOLS   R-squared (Between):              0.5949
No. Observations:                 553   R-squared (Within):               0.6403
Date:                Wed, May 06 2020   R-squared (Overall):              0.6036
Time:                        11:44:09   Log-likelihood                    169.00
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.321
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             74.321
                            

In [123]:
# Test transit fare revenue against TNC fare revenue

df['TNC_REVENUE_VS_TRANSIT_REVENUE']=df['TNC_REVENUE']/(df['FARE_ADJ_2018'])
df['TNC_COST_PER_TRIP']=df['TNC_REVENUE']/df['TNC_TRIPS']
df['TNC_COST_PER_TRIP']=df['TNC_COST_PER_TRIP'].fillna(0)
#df['TNC_COST_VS_TRANSIT_FARE']=df['TNC_COST_PER_TRIP']/(df['FARE_ADJ_2018']/df['UPT_ADJ'])
df['TNC_COST_VS_TRANSIT_FARE']=np.where(df['TNC_COST_PER_TRIP']==0,0,df['TNC_COST_PER_TRIP']/(df['FARE_ADJ_2018']/df['UPT_ADJ']))

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_VS_TRANSIT_REVENUE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6440
Estimator:                   PanelOLS   R-squared (Between):              0.5630
No. Observations:                 551   R-squared (Within):               0.6440
Date:                Wed, May 06 2020   R-squared (Overall):              0.5727
Time:                        11:44:10   Log-likelihood                    172.37
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      69.312
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.775   Distribution:                  F(13,498)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             69.312
                            

In [124]:

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6360
Estimator:                   PanelOLS   R-squared (Between):              0.6671
No. Observations:                 553   R-squared (Within):               0.6360
Date:                Wed, May 06 2020   R-squared (Overall):              0.6733
Time:                        11:44:11   Log-likelihood                    165.68
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      79.724
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(11,502)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             79.724
                            

In [125]:
#removed fare
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_VS_TRANSIT_FARE \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6344
Estimator:                   PanelOLS   R-squared (Between):              0.6278
No. Observations:                 553   R-squared (Within):               0.6344
Date:                Wed, May 06 2020   R-squared (Overall):              0.6351
Time:                        11:44:12   Log-likelihood                    164.48
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      87.272
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(10,503)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             87.272
                            

In [126]:
#TSD variable going the other way
df['TNC_COST_PER_TRIP_log']=np.log(df['TNC_COST_PER_TRIP']+1)

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6347
Estimator:                   PanelOLS   R-squared (Between):              0.6500
No. Observations:                 553   R-squared (Within):               0.6347
Date:                Wed, May 06 2020   R-squared (Overall):              0.6567
Time:                        11:44:13   Log-likelihood                    164.70
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      79.279
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(11,502)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             79.279
                            

In [127]:
df['TNC_COST_PER_TRIP_HI_OPEX']=df['TNC_COST_PER_TRIP']*df['HI_OPEX']
df['TNC_COST_PER_TRIP_MID_OPEX']=df['TNC_COST_PER_TRIP']*df['MID_OPEX']
df['TNC_COST_PER_TRIP_NY']=df['TNC_COST_PER_TRIP']*df['NEW_YORK']
df['TNC_COST_PER_TRIP_HINY']=df['TNC_COST_PER_TRIP_HI_OPEX']+df['TNC_COST_PER_TRIP_NY']

df['TNC_COST_PER_TRIP_HINY_log']=np.log(df['TNC_COST_PER_TRIP_HINY']+1)
df['TNC_COST_PER_TRIP_MID_OPEX_log']=np.log(df['TNC_COST_PER_TRIP_MID_OPEX']+1)

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6396
Estimator:                   PanelOLS   R-squared (Between):              0.5620
No. Observations:                 553   R-squared (Within):               0.6396
Date:                Wed, May 06 2020   R-squared (Overall):              0.5717
Time:                        11:44:13   Log-likelihood                    168.50
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.108
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             74.108
                            

In [128]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6397
Estimator:                   PanelOLS   R-squared (Between):              0.5814
No. Observations:                 553   R-squared (Within):               0.6397
Date:                Wed, May 06 2020   R-squared (Overall):              0.5903
Time:                        11:44:14   Log-likelihood                    168.55
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.129
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             74.129
                            

In [129]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6375
Estimator:                   PanelOLS   R-squared (Between):              0.5396
No. Observations:                 553   R-squared (Within):               0.6375
Date:                Wed, May 06 2020   R-squared (Overall):              0.5506
Time:                        11:44:15   Log-likelihood                    166.85
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.420
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.420
                            

In [130]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_WAGE_UNDER_1250_PCT \
                    + JTW_HOME_PCT \
                    + TNC_COST_PER_TRIP_HINY_log \
                    + TNC_COST_PER_TRIP_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6371
Estimator:                   PanelOLS   R-squared (Between):              0.5133
No. Observations:                 553   R-squared (Within):               0.6371
Date:                Wed, May 06 2020   R-squared (Overall):              0.5249
Time:                        11:44:15   Log-likelihood                    166.53
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.286
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.286
                            

In [131]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_RAIL_HINY \
                    + YEARS_SINCE_TNC_RAIL_MIDLOW \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6414
Estimator:                   PanelOLS   R-squared (Between):              0.4521
No. Observations:                 553   R-squared (Within):               0.6414
Date:                Wed, May 06 2020   R-squared (Overall):              0.4645
Time:                        11:44:16   Log-likelihood                    169.88
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.689
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             74.689
                            

In [132]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + POP_EMP_log \
                    + FARE_per_UPT_2018_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_RAIL_HINY \
                    + YEARS_SINCE_TNC_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6414
Estimator:                   PanelOLS   R-squared (Between):              0.4521
No. Observations:                 553   R-squared (Within):               0.6414
Date:                Wed, May 06 2020   R-squared (Overall):              0.4645
Time:                        11:44:16   Log-likelihood                    169.88
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.689
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             74.689
                            

In [133]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

# Look at the effect of years since 2012.
## Latest 7 time periods.

In [134]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['YEARS_AFTER_2010']>1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4639
Estimator:                   PanelOLS   R-squared (Between):              0.7553
No. Observations:                1770   R-squared (Within):               0.4639
Date:                Wed, May 06 2020   R-squared (Overall):              0.7556
Time:                        11:44:19   Log-likelihood                    2018.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      92.635
Entities:                         257   P-value                           0.0000
Avg Obs:                       6.8872   Distribution:                 F(14,1499)
Min Obs:                       1.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             92.635
                            

### Separate models by cluster

In [135]:
# combined model
#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5586
Estimator:                   PanelOLS   R-squared (Between):              0.9254
No. Observations:                3861   R-squared (Within):               0.5586
Date:                Wed, May 06 2020   R-squared (Overall):              0.9258
Time:                        11:44:21   Log-likelihood                    2398.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.46
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             324.46
                            

In [136]:
# high op-ex model

#new base model, for both modes

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['HI_OPEX']==1) | (df['NEW_YORK']==1)])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.7941
Estimator:                   PanelOLS   R-squared (Between):              0.3680
No. Observations:                 651   R-squared (Within):               0.7941
Date:                Wed, May 06 2020   R-squared (Overall):              0.3695
Time:                        11:44:21   Log-likelihood                    646.81
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      210.39
Entities:                          40   P-value                           0.0000
Avg Obs:                       16.275   Distribution:                  F(11,600)
Min Obs:                       9.0000                                           
Max Obs:                       17.000   F-statistic (robust):             210.39
                            

In [137]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['HI_OPEX']==1) | (df['NEW_YORK']==1)])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.7971
Estimator:                   PanelOLS   R-squared (Between):              0.4221
No. Observations:                 651   R-squared (Within):               0.7971
Date:                Wed, May 06 2020   R-squared (Overall):              0.4234
Time:                        11:44:22   Log-likelihood                    651.58
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      196.11
Entities:                          40   P-value                           0.0000
Avg Obs:                       16.275   Distribution:                  F(12,599)
Min Obs:                       9.0000                                           
Max Obs:                       17.000   F-statistic (robust):             196.11
                            

In [138]:
# mid op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_log \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['MID_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4953
Estimator:                   PanelOLS   R-squared (Between):              0.9754
No. Observations:                1280   R-squared (Within):               0.4953
Date:                Wed, May 06 2020   R-squared (Overall):              0.9750
Time:                        11:44:22   Log-likelihood                    758.83
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      105.74
Entities:                          84   P-value                           0.0000
Avg Obs:                       15.238   Distribution:                 F(11,1185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             105.74
                            

In [139]:
# low op-ex model
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5677
Estimator:                   PanelOLS   R-squared (Between):              0.9355
No. Observations:                1938   R-squared (Within):               0.5677
Date:                Wed, May 06 2020   R-squared (Overall):              0.9358
Time:                        11:44:23   Log-likelihood                    1184.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      235.76
Entities:                         133   P-value                           0.0000
Avg Obs:                       14.571   Distribution:                 F(10,1795)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             235.76
                            

In [140]:
## excluding low op-ex, since they are different

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==0])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5767
Estimator:                   PanelOLS   R-squared (Between):              0.9286
No. Observations:                1931   R-squared (Within):               0.5767
Date:                Wed, May 06 2020   R-squared (Overall):              0.9288
Time:                        11:44:23   Log-likelihood                    1282.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      203.82
Entities:                         124   P-value                           0.0000
Avg Obs:                       15.573   Distribution:                 F(12,1795)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             203.82
                            

In [141]:
# segment bus vs rail and low vs everything else
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6333
Estimator:                   PanelOLS   R-squared (Between):              0.6043
No. Observations:                 553   R-squared (Within):               0.6333
Date:                Wed, May 06 2020   R-squared (Overall):              0.6116
Time:                        11:44:24   Log-likelihood                    163.64
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      86.854
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(10,503)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             86.854
                            

In [142]:
# segment bus vs rail and low vs everything else
# bus -- mid and above

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==0])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5734
Estimator:                   PanelOLS   R-squared (Between):              0.9165
No. Observations:                1931   R-squared (Within):               0.5734
Date:                Wed, May 06 2020   R-squared (Overall):              0.9168
Time:                        11:44:24   Log-likelihood                    1275.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      241.59
Entities:                         124   P-value                           0.0000
Avg Obs:                       15.573   Distribution:                 F(10,1797)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             241.59
                            

In [143]:
# segment bus vs rail and low vs everything else
# bus -- low op-ex

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[(df['RAIL_FLAG']==0) & df['LOW_OPEX']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5677
Estimator:                   PanelOLS   R-squared (Between):              0.9355
No. Observations:                1938   R-squared (Within):               0.5677
Date:                Wed, May 06 2020   R-squared (Overall):              0.9358
Time:                        11:44:25   Log-likelihood                    1184.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      235.76
Entities:                         133   P-value                           0.0000
Avg Obs:                       14.571   Distribution:                 F(10,1795)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             235.76
                            

### Currently prefer the combined model

In [144]:
# combined model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5586
Estimator:                   PanelOLS   R-squared (Between):              0.9254
No. Observations:                3861   R-squared (Within):               0.5586
Date:                Wed, May 06 2020   R-squared (Overall):              0.9258
Time:                        11:44:26   Log-likelihood                    2398.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.46
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(14,3590)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             324.46
                            

In [145]:
# remove JTW

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5573
Estimator:                   PanelOLS   R-squared (Between):              0.9170
No. Observations:                3869   R-squared (Within):               0.5573
Date:                Wed, May 06 2020   R-squared (Overall):              0.9175
Time:                        11:44:27   Log-likelihood                    2399.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      348.47
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(13,3599)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             348.47
                            

In [146]:
# add maintenance
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5575
Estimator:                   PanelOLS   R-squared (Between):              0.9146
No. Observations:                3869   R-squared (Within):               0.5575
Date:                Wed, May 06 2020   R-squared (Overall):              0.9151
Time:                        11:44:27   Log-likelihood                    2400.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.79
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.79
                            

In [147]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Total_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5576
Estimator:                   PanelOLS   R-squared (Between):              0.9053
No. Observations:                3869   R-squared (Within):               0.5576
Date:                Wed, May 06 2020   R-squared (Overall):              0.9057
Time:                        11:44:28   Log-likelihood                    2400.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.94
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.94
                            

In [148]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5572
Estimator:                   PanelOLS   R-squared (Between):              0.9033
No. Observations:                3869   R-squared (Within):               0.5572
Date:                Wed, May 06 2020   R-squared (Overall):              0.9037
Time:                        11:44:28   Log-likelihood                    2398.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.41
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.41
                            

In [149]:
# maintenance variations
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical_log \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5573
Estimator:                   PanelOLS   R-squared (Between):              0.9043
No. Observations:                3869   R-squared (Within):               0.5573
Date:                Wed, May 06 2020   R-squared (Overall):              0.9047
Time:                        11:44:29   Log-likelihood                    2399.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.47
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.47
                            

In [150]:
# combined

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5574
Estimator:                   PanelOLS   R-squared (Between):              0.9148
No. Observations:                3869   R-squared (Within):               0.5574
Date:                Wed, May 06 2020   R-squared (Overall):              0.9153
Time:                        11:44:29   Log-likelihood                    2399.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.71
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.71
                            

In [151]:
# slightly less segmentation
df['TNC_REVENUE_BUS_MIDHINY'] = df['TNC_REVENUE_BUS_MID_OPEX'] + df['TNC_REVENUE_BUS_HINY']

In [152]:
# current winner

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY \
                    + TNC_REVENUE_BUS_LOW_OPEX \
                    + TNC_REVENUE_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5580
Estimator:                   PanelOLS   R-squared (Between):              0.9173
No. Observations:                3869   R-squared (Within):               0.5580
Date:                Wed, May 06 2020   R-squared (Overall):              0.9176
Time:                        11:44:29   Log-likelihood                    2402.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      349.57
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(13,3599)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             349.57
                            

In [153]:
#log transform
df['TNC_REVENUE_BUS_MIDHINY_log']=np.log(df['TNC_REVENUE_BUS_MIDHINY']+1)


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5574
Estimator:                   PanelOLS   R-squared (Between):              0.9148
No. Observations:                3869   R-squared (Within):               0.5574
Date:                Wed, May 06 2020   R-squared (Overall):              0.9153
Time:                        11:44:30   Log-likelihood                    2399.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      323.71
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(14,3598)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             323.71
                            

In [154]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_HINY_log \
                    + TNC_REVENUE_RAIL_HINY_log \
                    + TNC_REVENUE_BUS_MID_OPEX_log \
                    + TNC_REVENUE_RAIL_MID_OPEX_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + MDBF_Mechanical \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)



                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5589
Estimator:                   PanelOLS   R-squared (Between):              0.9115
No. Observations:                3869   R-squared (Within):               0.5589
Date:                Wed, May 06 2020   R-squared (Overall):              0.9119
Time:                        11:44:30   Log-likelihood                    2406.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      303.80
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.054   Distribution:                 F(15,3597)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             303.80
                            

### Bus vs rail with just TNCs segmented by op-ex

In [155]:
# bus

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5479
Estimator:                   PanelOLS   R-squared (Between):              0.9542
No. Observations:                3316   R-squared (Within):               0.5479
Date:                Wed, May 06 2020   R-squared (Overall):              0.9543
Time:                        11:44:32   Log-likelihood                    2429.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      340.21
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.281   Distribution:                 F(11,3088)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             340.21
                            

In [156]:
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6333
Estimator:                   PanelOLS   R-squared (Between):              0.6043
No. Observations:                 553   R-squared (Within):               0.6333
Date:                Wed, May 06 2020   R-squared (Overall):              0.6116
Time:                        11:44:32   Log-likelihood                    163.64
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      86.854
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(10,503)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             86.854
                            

### Current best models below, split bus vs rail

In [157]:
# bus - simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + TNC_REVENUE_BUS_MIDHINY_log \
                    + TNC_REVENUE_BUS_LOW_OPEX_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5479
Estimator:                   PanelOLS   R-squared (Between):              0.9542
No. Observations:                3316   R-squared (Within):               0.5479
Date:                Wed, May 06 2020   R-squared (Overall):              0.9543
Time:                        11:44:34   Log-likelihood                    2429.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      340.21
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.281   Distribution:                 F(11,3088)
Min Obs:                       6.0000                                           
Max Obs:                       17.000   F-statistic (robust):             340.21
                            

In [158]:
# rail -- simplified

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + TNC_REVENUE_RAIL_log \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6333
Estimator:                   PanelOLS   R-squared (Between):              0.6043
No. Observations:                 553   R-squared (Within):               0.6333
Date:                Wed, May 06 2020   R-squared (Overall):              0.6116
Time:                        11:44:34   Log-likelihood                    163.64
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      86.854
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(10,503)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             86.854
                            

# Models

#### Some additional Tests

In [159]:
# split by opex group
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5652
Estimator:                   PanelOLS   R-squared (Between):              0.9345
No. Observations:                3861   R-squared (Within):               0.5652
Date:                Wed, May 06 2020   R-squared (Overall):              0.9347
Time:                        11:44:36   Log-likelihood                    2427.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      310.99
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(15,3589)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             310.99
                            

In [160]:
# split bike and scooter by bus and rail
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE_BUS \
                    + BIKE_SHARE_RAIL \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5653
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                3861   R-squared (Within):               0.5653
Date:                Wed, May 06 2020   R-squared (Overall):              0.9349
Time:                        11:44:36   Log-likelihood                    2427.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      274.39
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(17,3587)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             274.39
                            

In [161]:
# only split scooter
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5653
Estimator:                   PanelOLS   R-squared (Between):              0.9341
No. Observations:                3861   R-squared (Within):               0.5653
Date:                Wed, May 06 2020   R-squared (Overall):              0.9343
Time:                        11:44:37   Log-likelihood                    2427.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.58
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.58
                            

In [162]:
# try emp percent
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5651
Estimator:                   PanelOLS   R-squared (Between):              0.9393
No. Observations:                3861   R-squared (Within):               0.5651
Date:                Wed, May 06 2020   R-squared (Overall):              0.9395
Time:                        11:44:38   Log-likelihood                    2426.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.33
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.33
                            

In [163]:
# TSD population and employment percent

# try emp percent
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5649
Estimator:                   PanelOLS   R-squared (Between):              0.9367
No. Observations:                3861   R-squared (Within):               0.5649
Date:                Wed, May 06 2020   R-squared (Overall):              0.9370
Time:                        11:44:38   Log-likelihood                    2425.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.13
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.13
                            

In [164]:

# TSD low income
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_1250 \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5651
Estimator:                   PanelOLS   R-squared (Between):              0.9422
No. Observations:                3861   R-squared (Within):               0.5651
Date:                Wed, May 06 2020   R-squared (Overall):              0.9424
Time:                        11:44:39   Log-likelihood                    2427.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.44
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.44
                            

In [165]:

# TSD low income
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_OVER_1250 \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5663
Estimator:                   PanelOLS   R-squared (Between):              0.9508
No. Observations:                3861   R-squared (Within):               0.5663
Date:                Wed, May 06 2020   R-squared (Overall):              0.9508
Time:                        11:44:40   Log-likelihood                    2432.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      292.81
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             292.81
                            

In [166]:

# TSD low income
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_WAGE_UNDER_3333 \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5650
Estimator:                   PanelOLS   R-squared (Between):              0.9399
No. Observations:                3861   R-squared (Within):               0.5650
Date:                Wed, May 06 2020   R-squared (Overall):              0.9401
Time:                        11:44:40   Log-likelihood                    2426.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.26
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.26
                            

In [167]:
# Drop TSD

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5626
Estimator:                   PanelOLS   R-squared (Between):              0.9392
No. Observations:                3974   R-squared (Within):               0.5626
Date:                Wed, May 06 2020   R-squared (Overall):              0.9394
Time:                        11:44:41   Log-likelihood                    2436.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      316.65
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(15,3693)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             316.65
                            

In [168]:
# split NY

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5630
Estimator:                   PanelOLS   R-squared (Between):              0.9404
No. Observations:                3974   R-squared (Within):               0.5630
Date:                Wed, May 06 2020   R-squared (Overall):              0.9406
Time:                        11:44:42   Log-likelihood                    2438.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.66
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.66
                            

In [169]:
# logs instead of linear on TNCS

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + np.log(1+PER_CAPITA_TNC_TRIPS_HINY_BUS) \
                    + np.log(1+PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS) \
                    + np.log(1+PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS) \
                    + np.log(1+PER_CAPITA_TNC_TRIPS_HINY_RAIL) \
                    + np.log(1+PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL) \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5617
Estimator:                   PanelOLS   R-squared (Between):              0.9358
No. Observations:                3974   R-squared (Within):               0.5617
Date:                Wed, May 06 2020   R-squared (Overall):              0.9360
Time:                        11:44:47   Log-likelihood                    2432.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      315.47
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(15,3693)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             315.47
                            

In [170]:
# bookings instead of trips

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_BOOKINGS_NEW_YORK_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_BOOKINGS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5629
Estimator:                   PanelOLS   R-squared (Between):              0.9402
No. Observations:                3974   R-squared (Within):               0.5629
Date:                Wed, May 06 2020   R-squared (Overall):              0.9403
Time:                        11:44:47   Log-likelihood                    2438.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.59
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.59
                            

In [171]:
# total trips instead of per-capita trips

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_BUS_NEW_YORK \
                    + TNC_TRIPS_BUS_HI_OPEX \
                    + TNC_TRIPS_BUS_MID_OPEX \
                    + TNC_TRIPS_BUS_LOW_OPEX \
                    + TNC_TRIPS_RAIL_NEW_YORK \
                    + TNC_TRIPS_RAIL_HI_OPEX \
                    + TNC_TRIPS_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5631
Estimator:                   PanelOLS   R-squared (Between):              0.9264
No. Observations:                3974   R-squared (Within):               0.5631
Date:                Wed, May 06 2020   R-squared (Overall):              0.9266
Time:                        11:44:48   Log-likelihood                    2439.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.84
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.84
                            

### Look at different splits

In [172]:
# try emp percent
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5651
Estimator:                   PanelOLS   R-squared (Between):              0.9393
No. Observations:                3861   R-squared (Within):               0.5651
Date:                Wed, May 06 2020   R-squared (Overall):              0.9395
Time:                        11:44:49   Log-likelihood                    2426.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      291.33
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(16,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             291.33
                            

In [173]:
# low op-ex bus
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5742
Estimator:                   PanelOLS   R-squared (Between):              0.9665
No. Observations:                1930   R-squared (Within):               0.5742
Date:                Wed, May 06 2020   R-squared (Overall):              0.9668
Time:                        11:44:50   Log-likelihood                    1192.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      218.91
Entities:                         133   P-value                           0.0000
Avg Obs:                       14.511   Distribution:                 F(11,1786)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             218.91
                            

In [174]:
# mid and high op-ex
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df[df['LOW_OPEX']!=1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5894
Estimator:                   PanelOLS   R-squared (Between):              0.9568
No. Observations:                1931   R-squared (Within):               0.5894
Date:                Wed, May 06 2020   R-squared (Overall):              0.9567
Time:                        11:44:51   Log-likelihood                    1311.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      171.47
Entities:                         124   P-value                           0.0000
Avg Obs:                       15.573   Distribution:                 F(15,1792)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             171.47
                            

In [175]:
# bus

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5596
Estimator:                   PanelOLS   R-squared (Between):              0.9825
No. Observations:                3308   R-squared (Within):               0.5596
Date:                Wed, May 06 2020   R-squared (Overall):              0.9826
Time:                        11:44:51   Log-likelihood                    2466.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      300.84
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(13,3078)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             300.84
                            

In [176]:
# rail

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_WA_EMP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HINY_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6377
Estimator:                   PanelOLS   R-squared (Between):              0.5737
No. Observations:                 553   R-squared (Within):               0.6377
Date:                Wed, May 06 2020   R-squared (Overall):              0.5829
Time:                        11:44:52   Log-likelihood                    167.05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      73.502
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(12,501)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             73.502
                            

## Combined Model

In [177]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HIMIDNY_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5595
Estimator:                   PanelOLS   R-squared (Between):              0.9124
No. Observations:                3861   R-squared (Within):               0.5595
Date:                Wed, May 06 2020   R-squared (Overall):              0.9128
Time:                        11:44:54   Log-likelihood                    2402.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      350.81
Entities:                         257   P-value                           0.0000
Avg Obs:                       15.023   Distribution:                 F(13,3591)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             350.81
                            

In [178]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['BUS_FLAG']==1])
res=mod.fit()
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5587
Estimator:                   PanelOLS   R-squared (Between):              0.9793
No. Observations:                3308   R-squared (Within):               0.5587
Date:                Wed, May 06 2020   R-squared (Overall):              0.9793
Time:                        11:44:54   Log-likelihood                    2462.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      278.28
Entities:                         217   P-value                           0.0000
Avg Obs:                       15.244   Distribution:                 F(14,3077)
Min Obs:                       4.0000                                           
Max Obs:                       17.000   F-statistic (robust):             278.28
                            

In [179]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_PCT_LEHD \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MIDLOW_RAIL \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag  \
                    + EntityEffects \
                    ',data=df[df['RAIL_FLAG']==1])
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.6386
Estimator:                   PanelOLS   R-squared (Between):              0.6463
No. Observations:                 553   R-squared (Within):               0.6386
Date:                Wed, May 06 2020   R-squared (Overall):              0.6531
Time:                        11:44:55   Log-likelihood                    167.71
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      67.968
Entities:                          40   P-value                           0.0000
Avg Obs:                       13.825   Distribution:                  F(13,500)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             67.968
                            

## Compare to years since ride-hail start

In [187]:
# base model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5630
Estimator:                   PanelOLS   R-squared (Between):              0.9404
No. Observations:                3974   R-squared (Within):               0.5630
Date:                Wed, May 06 2020   R-squared (Overall):              0.9406
Time:                        15:06:56   Log-likelihood                    2438.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.66
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.66
                            

In [184]:
# years since TNC

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS \
                    + YEARS_SINCE_TNC_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5603
Estimator:                   PanelOLS   R-squared (Between):              0.9330
No. Observations:                3974   R-squared (Within):               0.5603
Date:                Wed, May 06 2020   R-squared (Overall):              0.9331
Time:                        11:48:50   Log-likelihood                    2426.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      392.49
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(12,3696)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             392.49
                            

In [185]:
# years since TNC split


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_HI \
                    + YEARS_SINCE_TNC_BUS_MID \
                    + YEARS_SINCE_TNC_BUS_LOW \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_HI \
                    + YEARS_SINCE_TNC_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5630
Estimator:                   PanelOLS   R-squared (Between):              0.9387
No. Observations:                3974   R-squared (Within):               0.5630
Date:                Wed, May 06 2020   R-squared (Overall):              0.9389
Time:                        11:51:08   Log-likelihood                    2438.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.75
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.75
                            

In [224]:
df['FAV_EXT'] = 0
df['FAV_EXT'] = np.where(df['CLUSTER_APTA_EXTENDED'] % 10 == 1, 1, df['FAV_EXT'])
df['FAV_EXT'] = np.where(df['CLUSTER_APTA_EXTENDED'] % 10 == 2, 1, df['FAV_EXT'])
df['UNFAV_EXT'] = 1 - df['FAV_EXT']

df['STRONG_COMP'] = 0
df['STRONG_COMP'] = np.where(df['CLUSTER_APTA_EXTENDED'] % 10 == 1, 1, df['STRONG_COMP'])
df['STRONG_COMP'] = np.where(df['CLUSTER_APTA_EXTENDED'] % 10 == 3, 1, df['STRONG_COMP'])
                            
df['WEAK_COMP'] = 1 - df['STRONG_COMP']

In [225]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [228]:
# split by favorable vs unfavorable
df['YEARS_SINCE_TNC_BUS_HI_FAV']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['FAV_EXT']
df['YEARS_SINCE_TNC_BUS_MID_FAV'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_BUS_LOW_FAV'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_HI_FAV'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_MID_FAV']= df['YEARS_SINCE_TNC_RAIL_MID']* df['FAV_EXT']

df['YEARS_SINCE_TNC_BUS_HI_UNFAV']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['UNFAV_EXT']
df['YEARS_SINCE_TNC_BUS_MID_UNFAV'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_BUS_LOW_UNFAV'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_HI_UNFAV'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_MID_UNFAV']= df['YEARS_SINCE_TNC_RAIL_MID']* df['UNFAV_EXT']


df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['FAV_EXT']
# split by favorable vs unfavorable
df['YEARS_SINCE_TNC_BUS_HI_FAV']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['FAV_EXT']
df['YEARS_SINCE_TNC_BUS_MID_FAV'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_BUS_LOW_FAV'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_HI_FAV'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['FAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_MID_FAV']= df['YEARS_SINCE_TNC_RAIL_MID']* df['FAV_EXT']

df['YEARS_SINCE_TNC_BUS_HI_UNFAV']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['UNFAV_EXT']
df['YEARS_SINCE_TNC_BUS_MID_UNFAV'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_BUS_LOW_UNFAV'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_HI_UNFAV'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['UNFAV_EXT'] 
df['YEARS_SINCE_TNC_RAIL_MID_UNFAV']= df['YEARS_SINCE_TNC_RAIL_MID']* df['UNFAV_EXT']


df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['FAV_EXT']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['UNFAV_EXT']
#df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['FAV_EXT']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_HI_FAV \
                    + YEARS_SINCE_TNC_BUS_MID_FAV \
                    + YEARS_SINCE_TNC_BUS_LOW_FAV \
                    + YEARS_SINCE_TNC_BUS_HI_UNFAV \
                    + YEARS_SINCE_TNC_BUS_MID_UNFAV \
                    + YEARS_SINCE_TNC_BUS_LOW_UNFAV \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_HI_FAV \
                    + YEARS_SINCE_TNC_RAIL_MID_FAV \
                    + YEARS_SINCE_TNC_RAIL_HI_UNFAV \
                    + YEARS_SINCE_TNC_RAIL_MID_UNFAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 



                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5649
Estimator:                   PanelOLS   R-squared (Between):              0.9308
No. Observations:                3977   R-squared (Within):               0.5649
Date:                Fri, May 08 2020   R-squared (Overall):              0.9310
Time:                        14:53:12   Log-likelihood                    2448.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      217.65
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(22,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             217.65
                            

In [229]:
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['FAV_EXT']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['UNFAV_EXT']
#df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['FAV_EXT']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_UNFAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5642
Estimator:                   PanelOLS   R-squared (Between):              0.9464
No. Observations:                3977   R-squared (Within):               0.5642
Date:                Fri, May 08 2020   R-squared (Overall):              0.9466
Time:                        14:55:24   Log-likelihood                    2445.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      217.03
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(22,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             217.03
                            

In [236]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

#create FAC MAy 8

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5639
Estimator:                   PanelOLS   R-squared (Between):              0.9502
No. Observations:                3977   R-squared (Within):               0.5639
Date:                Fri, May 08 2020   R-squared (Overall):              0.9504
Time:                        15:21:40   Log-likelihood                    2443.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      238.52
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(20,3690)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             238.52
                            

In [237]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df[df['NEW_YORK']==0])
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5637
Estimator:                   PanelOLS   R-squared (Between):              0.9504
No. Observations:                3943   R-squared (Within):               0.5637
Date:                Fri, May 08 2020   R-squared (Overall):              0.9505
Time:                        15:23:39   Log-likelihood                    2410.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      262.68
Entities:                         265   P-value                           0.0000
Avg Obs:                       14.879   Distribution:                 F(18,3660)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             262.68
                            

In [238]:
list(df)

['CBSA',
 'UPT_ADJ',
 'VRM_ADJ',
 'FARE_ADJ',
 'FARE_per_UPT_cleaned',
 'min_year',
 'max_year',
 'FARE_per_UPT',
 'PROBLEM_FLAG',
 'PROBLEM_FLAG2',
 'PROBLEM_FLAG3',
 'Tot_Pop',
 'Tot_Instate_Pop',
 'Tot_Outstate_Pop',
 'Tot_NonUSA_POP',
 'Total_Median_Income_Individual',
 'Native_Instate_Med_Inc_Indiv',
 'Native_Outstate_Med_Inc_Indiv',
 'Total_Pop_Poverty',
 'Pop_Below100_Poverty',
 'Pop_Below150_Poverty',
 'Pop_Above150_Poverty',
 'Age_under18',
 'Age_18to64',
 'Age_over64',
 'Total_HH',
 'HH_0Veh',
 'HH_1Veh',
 'HH_2Veh',
 'HH_3Veh',
 'HH_4+Veh',
 'PCT_HH_NO_VEH',
 'HH_MED_INC',
 'HH_MEAN_INC',
 'INC_U35',
 'INC_35_100',
 'INC_100P',
 'TOT_UNEMP_MSA',
 'TOT_EMP_MSA',
 'TOT_LABOR_MSA',
 'UNEMP_RATE_PCT',
 'EMP_RATE_PCT',
 'GasPrice',
 'Area_acre',
 'POP_CENSUSTRACT',
 'AVG_SPEED',
 '2018_Dollar_Multiplier',
 'TNC_ARRIVAL',
 'TNC_FLAG',
 'YEARS_SINCE_TNC',
 'FARE_ADJ_2018',
 'TOTAL_MED_INC_INDIV_2018',
 'NATIVE_INSTATE_MED_INC_2018',
 'NATIVE_OUTSTATE_MED_INC_2018',
 'HH_MED_INC_201

In [241]:
df['YEARS_SINCE_TNC_NEW_YORK_RAIL']=df['YEARS_SINCE_TNC']*df['NEW_YORK']*df['RAIL_FLAG']
df['YEARS_SINCE_TNC_NEW_YORK_BUS']=df['YEARS_SINCE_TNC']*df['NEW_YORK']*df['BUS_FLAG']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_NEW_YORK_BUS \
                    + YEARS_SINCE_TNC_BUS_HI_FAV \
                    + YEARS_SINCE_TNC_BUS_MID_FAV \
                    + YEARS_SINCE_TNC_BUS_LOW_FAV \
                    + YEARS_SINCE_TNC_BUS_HI_UNFAV \
                    + YEARS_SINCE_TNC_BUS_MID_UNFAV \
                    + YEARS_SINCE_TNC_BUS_LOW_UNFAV \
                    + YEARS_SINCE_TNC_NEW_YORK_RAIL \
                    + YEARS_SINCE_TNC_RAIL_HI_FAV \
                    + YEARS_SINCE_TNC_RAIL_MID_FAV \
                    + YEARS_SINCE_TNC_RAIL_HI_UNFAV \
                    + YEARS_SINCE_TNC_RAIL_MID_UNFAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

#create FAC charts May 8

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5649
Estimator:                   PanelOLS   R-squared (Between):              0.9308
No. Observations:                3977   R-squared (Within):               0.5649
Date:                Fri, May 08 2020   R-squared (Overall):              0.9310
Time:                        15:27:49   Log-likelihood                    2448.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      217.65
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(22,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             217.65
                            

In [233]:
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['STRONG_COMP']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['FAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['STRONG_COMP']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5600
Estimator:                   PanelOLS   R-squared (Between):              0.9322
No. Observations:                3977   R-squared (Within):               0.5600
Date:                Fri, May 08 2020   R-squared (Overall):              0.9324
Time:                        15:07:14   Log-likelihood                    2426.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      293.89
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(16,3694)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             293.89
                            

In [234]:
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['UNFAV_EXT']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['UNFAV_EXT']*df['WEAK_COMP']


df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['FAV_EXT']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS']*df['FAV_EXT']*df['WEAK_COMP']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['UNFAV_EXT']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['UNFAV_EXT']*df['WEAK_COMP']


df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['FAV_EXT']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL']*df['FAV_EXT']*df['WEAK_COMP']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_FAV_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_UNFAV_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_FAV_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_UNFAV_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5632
Estimator:                   PanelOLS   R-squared (Between):              0.9312
No. Observations:                3977   R-squared (Within):               0.5632
Date:                Fri, May 08 2020   R-squared (Overall):              0.9314
Time:                        15:11:46   Log-likelihood                    2440.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      237.88
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(20,3690)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             237.88
                            

In [235]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5611
Estimator:                   PanelOLS   R-squared (Between):              0.9321
No. Observations:                3977   R-squared (Within):               0.5611
Date:                Fri, May 08 2020   R-squared (Overall):              0.9323
Time:                        15:16:11   Log-likelihood                    2431.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      295.16
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(16,3694)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             295.16
                            

In [230]:
# split by weak vs strong competitiveness
df['YEARS_SINCE_TNC_BUS_HI_STRONG']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['STRONG_COMP']
df['YEARS_SINCE_TNC_BUS_MID_STRONG'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['STRONG_COMP'] 
df['YEARS_SINCE_TNC_BUS_LOW_STRONG'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['STRONG_COMP'] 
df['YEARS_SINCE_TNC_RAIL_HI_STRONG'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['STRONG_COMP'] 
df['YEARS_SINCE_TNC_RAIL_MID_STRONG']= df['YEARS_SINCE_TNC_RAIL_MID']* df['STRONG_COMP']

df['YEARS_SINCE_TNC_BUS_HI_WEAK']  = df['YEARS_SINCE_TNC_BUS_HI']  * df['WEAK_COMP']
df['YEARS_SINCE_TNC_BUS_MID_WEAK'] = df['YEARS_SINCE_TNC_BUS_MID'] * df['WEAK_COMP'] 
df['YEARS_SINCE_TNC_BUS_LOW_WEAK'] = df['YEARS_SINCE_TNC_BUS_LOW'] * df['WEAK_COMP'] 
df['YEARS_SINCE_TNC_RAIL_HI_WEAK'] = df['YEARS_SINCE_TNC_RAIL_HI'] * df['WEAK_COMP'] 
df['YEARS_SINCE_TNC_RAIL_MID_WEAK']= df['YEARS_SINCE_TNC_RAIL_MID']* df['WEAK_COMP']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['STRONG_COMP']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['WEAK_COMP']
#df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['STRONG_COMP']
    
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_HI_STRONG \
                    + YEARS_SINCE_TNC_BUS_MID_STRONG \
                    + YEARS_SINCE_TNC_BUS_LOW_STRONG \
                    + YEARS_SINCE_TNC_BUS_HI_WEAK \
                    + YEARS_SINCE_TNC_BUS_MID_WEAK \
                    + YEARS_SINCE_TNC_BUS_LOW_WEAK \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_HI_STRONG \
                    + YEARS_SINCE_TNC_RAIL_MID_STRONG \
                    + YEARS_SINCE_TNC_RAIL_HI_WEAK \
                    + YEARS_SINCE_TNC_RAIL_MID_WEAK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5660
Estimator:                   PanelOLS   R-squared (Between):              0.9370
No. Observations:                3977   R-squared (Within):               0.5660
Date:                Fri, May 08 2020   R-squared (Overall):              0.9371
Time:                        14:58:39   Log-likelihood                    2453.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      218.65
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(22,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             218.65
                            

In [231]:
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['STRONG_COMP']

df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['WEAK_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_WEAK']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['WEAK_COMP']
#df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_LOW_OPEX_UNFAV']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX']*df['UNFAV_EXT']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX']*df['STRONG_COMP']
df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_STRONG']=df['TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX']*df['STRONG_COMP']
    

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_WEAK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX_STRONG \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5662
Estimator:                   PanelOLS   R-squared (Between):              0.9506
No. Observations:                3977   R-squared (Within):               0.5662
Date:                Fri, May 08 2020   R-squared (Overall):              0.9507
Time:                        15:00:34   Log-likelihood                    2454.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      209.22
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(23,3687)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             209.22
                            

In [ ]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_HI_STRONG \
                    + YEARS_SINCE_TNC_BUS_MID_STRONG \
                    + YEARS_SINCE_TNC_BUS_LOW_STRONG \
                    + YEARS_SINCE_TNC_BUS_HI_WEAK \
                    + YEARS_SINCE_TNC_BUS_MID_WEAK \
                    + YEARS_SINCE_TNC_BUS_LOW_WEAK \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_HI_STRONG \
                    + YEARS_SINCE_TNC_RAIL_MID_STRONG \
                    + YEARS_SINCE_TNC_RAIL_HI_WEAK \
                    + YEARS_SINCE_TNC_RAIL_MID_WEAK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

In [200]:
# split by weak vs strong competitiveness - simplified
df['YEARS_SINCE_TNC_BUS_STRONG']  = df['YEARS_SINCE_TNC_BUS']  * df['STRONG_COMP']
df['YEARS_SINCE_TNC_RAIL_STRONG'] = df['YEARS_SINCE_TNC_RAIL'] * df['STRONG_COMP'] 

df['YEARS_SINCE_TNC_BUS_WEAK']  = df['YEARS_SINCE_TNC_BUS']  * df['WEAK_COMP']
df['YEARS_SINCE_TNC_RAIL_WEAK'] = df['YEARS_SINCE_TNC_RAIL'] * df['WEAK_COMP'] 
    
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_STRONG \
                    + YEARS_SINCE_TNC_BUS_WEAK \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_STRONG \
                    + YEARS_SINCE_TNC_RAIL_WEAK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5623
Estimator:                   PanelOLS   R-squared (Between):              0.9368
No. Observations:                3974   R-squared (Within):               0.5623
Date:                Wed, May 06 2020   R-squared (Overall):              0.9369
Time:                        16:14:56   Log-likelihood                    2435.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      296.41
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(16,3692)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             296.41
                            

In [201]:
# split by favorable vs unfavorable - simplified
df['YEARS_SINCE_TNC_BUS_FAV']  = df['YEARS_SINCE_TNC_BUS']  * df['FAV_EXT']
df['YEARS_SINCE_TNC_RAIL_FAV'] = df['YEARS_SINCE_TNC_RAIL'] * df['FAV_EXT'] 

df['YEARS_SINCE_TNC_BUS_UNFAV']  = df['YEARS_SINCE_TNC_BUS']  * df['UNFAV_EXT']
df['YEARS_SINCE_TNC_RAIL_UNFAV'] = df['YEARS_SINCE_TNC_RAIL'] * df['UNFAV_EXT'] 
    
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_FAV \
                    + YEARS_SINCE_TNC_BUS_UNFAV \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_FAV \
                    + YEARS_SINCE_TNC_RAIL_UNFAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5623
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                3974   R-squared (Within):               0.5623
Date:                Wed, May 06 2020   R-squared (Overall):              0.9349
Time:                        16:15:34   Log-likelihood                    2435.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      296.45
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(16,3692)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             296.45
                            

In [202]:
# split by four segments
df['YEARS_SINCE_TNC_BUS_FAV_STRONG']  = df['YEARS_SINCE_TNC_BUS']  * df['FAV_EXT'] * df['STRONG_COMP']
df['YEARS_SINCE_TNC_RAIL_FAV_STRONG'] = df['YEARS_SINCE_TNC_RAIL'] * df['FAV_EXT'] * df['STRONG_COMP']

df['YEARS_SINCE_TNC_BUS_UNFAV_STRONG']  = df['YEARS_SINCE_TNC_BUS']  * df['UNFAV_EXT'] * df['STRONG_COMP']
df['YEARS_SINCE_TNC_RAIL_UNFAV_STRONG'] = df['YEARS_SINCE_TNC_RAIL'] * df['UNFAV_EXT'] * df['STRONG_COMP'] 
    
df['YEARS_SINCE_TNC_BUS_FAV_WEAK']  = df['YEARS_SINCE_TNC_BUS']  * df['FAV_EXT'] * df['WEAK_COMP']
df['YEARS_SINCE_TNC_RAIL_FAV_WEAK'] = df['YEARS_SINCE_TNC_RAIL'] * df['FAV_EXT'] * df['WEAK_COMP']

df['YEARS_SINCE_TNC_BUS_UNFAV_WEAK']  = df['YEARS_SINCE_TNC_BUS']  * df['UNFAV_EXT'] * df['WEAK_COMP']
df['YEARS_SINCE_TNC_RAIL_UNFAV_WEAK'] = df['YEARS_SINCE_TNC_RAIL'] * df['UNFAV_EXT'] * df['WEAK_COMP']

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_BUS_NY \
                    + YEARS_SINCE_TNC_BUS_FAV_STRONG \
                    + YEARS_SINCE_TNC_BUS_UNFAV_STRONG \
                    + YEARS_SINCE_TNC_BUS_FAV_WEAK \
                    + YEARS_SINCE_TNC_BUS_UNFAV_WEAK \
                    + YEARS_SINCE_TNC_RAIL_NY \
                    + YEARS_SINCE_TNC_RAIL_FAV_STRONG \
                    + YEARS_SINCE_TNC_RAIL_UNFAV_STRONG \
                    + YEARS_SINCE_TNC_RAIL_FAV_WEAK \
                    + YEARS_SINCE_TNC_RAIL_UNFAV_WEAK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5648
Estimator:                   PanelOLS   R-squared (Between):              0.9318
No. Observations:                3974   R-squared (Within):               0.5648
Date:                Wed, May 06 2020   R-squared (Overall):              0.9319
Time:                        16:15:56   Log-likelihood                    2446.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      239.27
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(20,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             239.27
                            

### TNC trips by different clusters

In [203]:
# base model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_LOW_OPEX_BUS \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + PER_CAPITA_TNC_TRIPS_HI_OPEX_RAIL \
                    + PER_CAPITA_TNC_TRIPS_MID_OPEX_RAIL \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5630
Estimator:                   PanelOLS   R-squared (Between):              0.9404
No. Observations:                3974   R-squared (Within):               0.5630
Date:                Wed, May 06 2020   R-squared (Overall):              0.9406
Time:                        16:18:52   Log-likelihood                    2438.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      279.66
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             279.66
                            

In [204]:

df['PER_CAPITA_TNC_TRIPS_BUS_FAV_STRONG']  = df['PER_CAPITA_TNC_TRIPS_BUS']  * df['FAV_EXT'] * df['STRONG_COMP']
df['PER_CAPITA_TNC_TRIPS_RAIL_FAV_STRONG'] = df['PER_CAPITA_TNC_TRIPS_RAIL'] * df['FAV_EXT'] * df['STRONG_COMP']

df['PER_CAPITA_TNC_TRIPS_BUS_UNFAV_STRONG']  = df['PER_CAPITA_TNC_TRIPS_BUS']  * df['UNFAV_EXT'] * df['STRONG_COMP']
df['PER_CAPITA_TNC_TRIPS_RAIL_UNFAV_STRONG'] = df['PER_CAPITA_TNC_TRIPS_RAIL'] * df['UNFAV_EXT'] * df['STRONG_COMP'] 
    
df['PER_CAPITA_TNC_TRIPS_BUS_FAV_WEAK']  = df['PER_CAPITA_TNC_TRIPS_BUS']  * df['FAV_EXT'] * df['WEAK_COMP']
df['PER_CAPITA_TNC_TRIPS_RAIL_FAV_WEAK'] = df['PER_CAPITA_TNC_TRIPS_RAIL'] * df['FAV_EXT'] * df['WEAK_COMP']

df['PER_CAPITA_TNC_TRIPS_BUS_UNFAV_WEAK']  = df['PER_CAPITA_TNC_TRIPS_BUS']  * df['UNFAV_EXT'] * df['WEAK_COMP']
df['PER_CAPITA_TNC_TRIPS_RAIL_UNFAV_WEAK'] = df['PER_CAPITA_TNC_TRIPS_RAIL'] * df['UNFAV_EXT'] * df['WEAK_COMP']


In [207]:
# base model

mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_BUS \
                    + PER_CAPITA_TNC_TRIPS_BUS_FAV_STRONG \
                    + PER_CAPITA_TNC_TRIPS_BUS_UNFAV_STRONG \
                    + PER_CAPITA_TNC_TRIPS_BUS_FAV_WEAK \
                    + PER_CAPITA_TNC_TRIPS_BUS_UNFAV_WEAK \
                    + PER_CAPITA_TNC_TRIPS_NEW_YORK_RAIL \
                    + PER_CAPITA_TNC_TRIPS_RAIL_FAV_STRONG \
                    + PER_CAPITA_TNC_TRIPS_RAIL_UNFAV_STRONG \
                    + PER_CAPITA_TNC_TRIPS_RAIL_FAV_WEAK \
                    + PER_CAPITA_TNC_TRIPS_RAIL_UNFAV_WEAK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5663
Estimator:                   PanelOLS   R-squared (Between):              0.9118
No. Observations:                3974   R-squared (Within):               0.5663
Date:                Wed, May 06 2020   R-squared (Overall):              0.9120
Time:                        16:22:30   Log-likelihood                    2453.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      240.82
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(20,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             240.82
                            

In [190]:
# TNC Trips per transit trip

df['YEARS_SINCE_TNC_BUS_HI']  = df['YEARS_SINCE_TNC_BUS'] * df['HI_OPEX']
df['YEARS_SINCE_TNC_BUS_MID'] = df['YEARS_SINCE_TNC_BUS'] * df['MID_OPEX']
df['YEARS_SINCE_TNC_BUS_LOW'] = df['YEARS_SINCE_TNC_BUS'] * df['LOW_OPEX']
df['YEARS_SINCE_TNC_BUS_NY']  = df['YEARS_SINCE_TNC_BUS'] * df['NEW_YORK']

df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']=df['TNC_TRIPS']/df['UPT_ADJ']

df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_NY']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']*df['NEW_YORK']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_HI']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']*df['HI_OPEX']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_MID']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']*df['MID_OPEX']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_LOW']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['BUS_FLAG']*df['LOW_OPEX']

df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_NY']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']*df['NEW_YORK']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_HI']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']*df['HI_OPEX']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_MID']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']*df['MID_OPEX']
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_LOW']=df['TNC_TRIP_VS_TRANSIT_RIDERSHIP']*df['RAIL_FLAG']*df['LOW_OPEX']


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_NY \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_HI \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_MID \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_LOW \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_NY \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_HI \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5601
Estimator:                   PanelOLS   R-squared (Between):              0.9128
No. Observations:                3974   R-squared (Within):               0.5601
Date:                Wed, May 06 2020   R-squared (Overall):              0.9131
Time:                        15:17:43   Log-likelihood                    2425.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      276.43
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             276.43
                            

In [191]:
# TNC Trips per transit trip

df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_NY_log'] =np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_NY'] )
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_HI_log'] =np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_HI'] )
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_MID_log']=np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_MID'])
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_LOW_log']=np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_LOW'])

df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_NY_log'] =np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_NY'] )
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_HI_log'] =np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_HI'] )
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_MID_log']=np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_MID'])
df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_LOW_log']=np.log(1+df['TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_LOW'])


mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_NY_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_HI_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_MID_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_BUS_LOW_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_NY_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_HI_log \
                    + TNC_TRIP_VS_TRANSIT_RIDERSHIP_RAIL_MID_log \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5610
Estimator:                   PanelOLS   R-squared (Between):              0.9233
No. Observations:                3974   R-squared (Within):               0.5610
Date:                Wed, May 06 2020   R-squared (Overall):              0.9235
Time:                        15:20:49   Log-likelihood                    2429.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      277.43
Entities:                         266   P-value                           0.0000
Avg Obs:                       14.940   Distribution:                 F(17,3691)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             277.43
                            

In [ ]:
# more segmentation on years since TNC


# FAC Charts

In [323]:
#model 1
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_NEW_YORK_BUS \
                    + YEARS_SINCE_TNC_BUS_HI_FAV \
                    + YEARS_SINCE_TNC_BUS_MID_FAV \
                    + YEARS_SINCE_TNC_BUS_LOW_FAV \
                    + YEARS_SINCE_TNC_BUS_HI_UNFAV \
                    + YEARS_SINCE_TNC_BUS_MID_UNFAV \
                    + YEARS_SINCE_TNC_BUS_LOW_UNFAV \
                    + YEARS_SINCE_TNC_NEW_YORK_RAIL \
                    + YEARS_SINCE_TNC_RAIL_HI_FAV \
                    + YEARS_SINCE_TNC_RAIL_MID_FAV \
                    + YEARS_SINCE_TNC_RAIL_HI_UNFAV \
                    + YEARS_SINCE_TNC_RAIL_MID_UNFAV \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5649
Estimator:                   PanelOLS   R-squared (Between):              0.9308
No. Observations:                3977   R-squared (Within):               0.5649
Date:                Tue, May 12 2020   R-squared (Overall):              0.9310
Time:                        10:44:40   Log-likelihood                    2448.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      217.65
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(22,3688)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             217.65
                            

In [335]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_NEW_YORK_BUS \
                    + YEARS_SINCE_TNC_BUS_HI_FAV \
                    + YEARS_SINCE_TNC_BUS_MID_FAV \
                    + YEARS_SINCE_TNC_BUS_LOW_FAV \
                    + YEARS_SINCE_TNC_BUS_HI_UNFAV \
                    + YEARS_SINCE_TNC_BUS_MID_UNFAV \
                    + YEARS_SINCE_TNC_BUS_LOW_UNFAV \
                    + YEARS_SINCE_TNC_NEW_YORK_RAIL \
                    + YEARS_SINCE_TNC_RAIL_HI \
                    + YEARS_SINCE_TNC_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5639
Estimator:                   PanelOLS   R-squared (Between):              0.9370
No. Observations:                3977   R-squared (Within):               0.5639
Date:                Tue, May 12 2020   R-squared (Overall):              0.9372
Time:                        11:04:32   Log-likelihood                    2444.1
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      238.59
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(20,3690)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             238.59
                            

In [351]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + YEARS_SINCE_TNC_NEW_YORK_BUS \
                    + YEARS_SINCE_TNC_BUS_HI_FAV \
                    + YEARS_SINCE_TNC_BUS_MID_FAV \
                    + YEARS_SINCE_TNC_BUS_LOW_FAV \
                    + YEARS_SINCE_TNC_BUS_HI_UNFAV \
                    + YEARS_SINCE_TNC_BUS_MID_UNFAV \
                    + YEARS_SINCE_TNC_BUS_LOW_UNFAV \
                    + YEARS_SINCE_TNC_NEW_YORK_RAIL \
                    + YEARS_SINCE_TNC_RAIL_HI \
                    + YEARS_SINCE_TNC_RAIL_MID \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5662
Estimator:                   PanelOLS   R-squared (Between):              0.9347
No. Observations:                3864   R-squared (Within):               0.5662
Date:                Wed, May 13 2020   R-squared (Overall):              0.9350
Time:                        11:33:15   Log-likelihood                    2433.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      222.82
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(21,3585)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             222.82
                            

In [256]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res)
#MAY 8

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5653
Estimator:                   PanelOLS   R-squared (Between):              0.9493
No. Observations:                3864   R-squared (Within):               0.5653
Date:                Mon, May 11 2020   R-squared (Overall):              0.9495
Time:                        01:52:52   Log-likelihood                    2428.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      259.20
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(18,3588)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             259.20
                            

In [305]:
df['CLUSTER_APTA_EXTENDED'].unique()

array([ 24,  34,  23,  22,  32,  31,  33,  14,  11,  21,  12, 100,  13])

In [306]:
df['FAV_EXT']=df['FAV_EXT'].astype(str)
df['CLUSTER_APTA_HI_MID_LOW_NY']=df['CLUSTER_APTA_EXTENDED'].apply(lambda x: str(x/10)[0])

def cluster_apta(df):
    x=str(df['CLUSTER_APTA_EXTENDED']/10)[0]
    if df['CLUSTER_APTA_EXTENDED']==100:
        return '10'
    elif (df['CLUSTER_APTA_EXTENDED']%10==1)|(df['CLUSTER_APTA_EXTENDED']%10==2):
        return x+'1'
    else: 
        return x+'2'
    
df['CLUSTER_APTA_FAV_UNFAV']=df.apply(cluster_apta,axis=1)

In [307]:
df['CLUSTER_APTA_FAV_UNFAV'].unique()

array(['22', '32', '21', '31', '12', '11', '10'], dtype=object)

In [337]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5639
Estimator:                   PanelOLS   R-squared (Between):              0.9502
No. Observations:                3977   R-squared (Within):               0.5639
Date:                Wed, May 13 2020   R-squared (Overall):              0.9504
Time:                        11:24:07   Log-likelihood                    2443.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      238.52
Entities:                         267   P-value                           0.0000
Avg Obs:                       14.895   Distribution:                 F(20,3690)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             238.52
                            

In [338]:
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_2018_log \
                    + POP_EMP_log \
                    + GAS_PRICE_2018_log \
                    + TOTAL_MED_INC_INDIV_2018_log \
                    + PCT_HH_NO_VEH \
                    + TSD_POP_EMP_PCT \
                    + JTW_HOME_PCT \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_FAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_HI_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_MID_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_BUS_LOW_OPEX_UNFAV \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_NEW_YORK \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_HI_OPEX \
                    + TNC_TRIPS_PER_CAPITA_CLUSTER_RAIL_MID_OPEX \
                    + BIKE_SHARE \
                    + scooter_flag_BUS  \
                    + scooter_flag_RAIL  \
                    + EntityEffects \
                    ',data=df)
res=mod.fit()
print(res) 

                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5660
Estimator:                   PanelOLS   R-squared (Between):              0.9484
No. Observations:                3864   R-squared (Within):               0.5660
Date:                Wed, May 13 2020   R-squared (Overall):              0.9487
Time:                        11:24:16   Log-likelihood                    2432.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      222.62
Entities:                         258   P-value                           0.0000
Avg Obs:                       14.977   Distribution:                 F(21,3585)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             222.62
                            

In [352]:
# keep the relevant fields in the data set
app_data = pd.concat([df[['RAIL_FLAG', 'CLUSTER_APTA_FAV_UNFAV']], 
                      res.model.dependent.dataframe, 
                      res.model.exog.dataframe, 
                      res.estimated_effects, 
                      res.resids], axis=1)

In [353]:
# calculate the fitted values
app_data['fitted'] = app_data['estimated_effects']
for var in res.params.keys(): 
    app_data['fitted'] = app_data['fitted'] + (res.params[var] * app_data[var])

# and check that we add up correctly
app_data['check_fitted'] = app_data['fitted'] + app_data['residual'] - app_data['UPT_ADJ_log']

In [354]:
# create the necessary columns

# dependents
app_data['UPT_ADJ']    = np.exp(app_data['UPT_ADJ_log']) - 1
app_data['fitted_exp'] = np.exp(app_data['fitted']) - 1

# linear versions of logged variables
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '')] = np.exp(app_data[var]) - 1              

# ratios and differences
for var in res.params.keys(): 
    if '_log' in var: 
        app_data[var.replace('_log', '_ratio')] = 1
    else: 
        app_data[var+'_diff'] = 0

# FAC multipliers
for var in res.params.keys(): 
    app_data[var+'_FAC_ratio'] = 1

app_data['UPT_ADJ_base'] = 0
app_data['UPT_ADJ_ratio'] = 1
app_data['fitted_exp_ratio'] = 1
app_data['UPT_ADJ_diff'] = 0
app_data['fitted_exp_diff'] = 0

# for tracking new systems
app_data['UPT_ADJ_first_year'] = 0
app_data['UPT_ADJ_new_reporter'] = 0

In [355]:
# get the ridership in the first year the system starts. This allows us to track new systems. 
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()
    first_year = years[0]

    app_data.loc[(id,first_year),'UPT_ADJ_new_reporter'] = app_data.loc[(id,first_year),'UPT_ADJ']
    
    for year in years: 
        app_data.loc[(id,year),'UPT_ADJ_first_year']   = app_data.loc[(id,first_year),'UPT_ADJ']

In [356]:
# calculate the FAC relative to a specific base year
ids = app_data.index.get_level_values(0).unique()

for id in ids: 
    years = app_data.loc[id].index.get_level_values(0).sort_values()

    for year in years[1:]:       
        base_year = year-1
        
        for var in res.params.keys(): 
            # ratios and differences
            if '_log' in var: 
                out_var = var.replace('_log', '_ratio')
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 / app_data.loc[(id,base_year), var])
            else: 
                out_var = var+'_diff'                    
                app_data.loc[(id,year), out_var] = (app_data.loc[(id,year), var] 
                                                 - app_data.loc[(id,base_year), var])

            # FAC multipliers
            app_data.loc[(id,year),var+'_FAC_ratio'] = np.exp(res.params[var] * (
                                                    app_data.loc[(id,year), var] 
                                                    - app_data.loc[(id,base_year), var]))

            # estimated effects (if time effects is zero, no change)
            app_data.loc[(id,year),'effects_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'estimated_effects'] 
                                                    - app_data.loc[(id,base_year), 'estimated_effects'])

            # residual
            app_data.loc[(id,year),'residual_FAC_ratio'] = np.exp(
                                                    app_data.loc[(id,year), 'residual'] 
                                                    - app_data.loc[(id,base_year), 'residual'])

        # observed and fitted changes            
        app_data.loc[(id,year),'UPT_ADJ_base'] = app_data.loc[(id,base_year),'UPT_ADJ']
        app_data.loc[(id,year),'UPT_ADJ_ratio'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 / app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_ratio'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 / app_data.loc[(id,base_year),'fitted_exp'])

        app_data.loc[(id,year),'UPT_ADJ_diff'] = (app_data.loc[(id,year),'UPT_ADJ'] 
                                                 - app_data.loc[(id,base_year),'UPT_ADJ'])
        app_data.loc[(id,year),'fitted_exp_diff'] = (app_data.loc[(id,year),'fitted_exp'] 
                                                 - app_data.loc[(id,base_year),'fitted_exp'])

/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/Users/rianajawad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app


In [357]:
# calculate the FAC
app_data['FAC_Sum'] = 0
for var in res.params.keys(): 
    app_data[var+'_FAC'] = (app_data[var+'_FAC_ratio'] - 1) * app_data['UPT_ADJ_base']
    app_data['FAC_Sum'] = app_data['FAC_Sum'] + app_data[var+'_FAC']




In [358]:

# total FAC is based on the fitted model (applied multiplicitively)
app_data['Known_FAC'] = (app_data['fitted_exp_ratio'] - 1) * app_data['UPT_ADJ_base']

# uknown change is the difference between the observed change and the known change
app_data['Unknown_FAC'] = app_data['UPT_ADJ_diff'] - app_data['Known_FAC']

# the change in ridership associated new systems
app_data['New_Reporter_FAC'] = app_data['UPT_ADJ_new_reporter'] 

# should be teh same as UPT_ADJ_DIFF
app_data['Total_Change'] = app_data['Known_FAC'] + app_data['Unknown_FAC'] + app_data['New_Reporter_FAC']

In [360]:
# calculate APTA - 4 groups
#app_data['CLUSTER_APTA4'] = np.floor(app_data['CLUSTER_APTA']/10)

In [359]:
# reset the index so I get ID and year
app_data = app_data.reset_index()
app_data = app_data.rename(columns={'level_0' : 'ID', 'level_1' : 'Year'})
app_data.to_csv('FAC_10-3-Model_1_2.csv')

In [361]:
# these are the fields we keep
out_fields = ['ID', 'Year', 'RAIL_FLAG', 'CLUSTER_APTA_FAV_UNFAV',    
              'UPT_ADJ_first_year', 
              'UPT_ADJ_base', 'UPT_ADJ', 'UPT_ADJ_diff', 
              'fitted_exp', 'fitted_exp_diff']

# exogenous variables 
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    out_fields = out_fields + [out_var]
    
# FAC
for var in res.params.keys(): 
    out_fields = out_fields + [var+'_FAC']
out_fields = out_fields + ['FAC_Sum', 'Known_FAC', 'Unknown_FAC', 'New_Reporter_FAC', 'Total_Change']
    
# keep those fields
summary_data = app_data.reset_index()[out_fields]

In [362]:
# take weighted average of exogenous variables
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data[out_var] = summary_data[out_var] * summary_data['UPT_ADJ_first_year']

# aggregate to bus/rail totals
summary_data_apta4 = summary_data.groupby(by=['CLUSTER_APTA_FAV_UNFAV', 'RAIL_FLAG', 'Year']).agg('sum')

# divide for weighted averages
for var in res.params.keys(): 
    out_var = var.replace('_log', '')
    summary_data_apta4[out_var] = summary_data_apta4[out_var] / summary_data_apta4['UPT_ADJ_first_year']

In [363]:

summary_data_apta4.to_csv('FAC_totals_APTA4_CLUSTERS_Model_1_2.csv')




In [285]:
df['CLUSTER_APTA_EXTENDED'].unique()

array([ 24,  34,  23,  22,  32,  31,  33,  14,  11,  21,  12, 100,  13])